In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from matplotlib import cbook, rc_params_from_file, rcParamsDefault
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score                               
from sklearn import metrics

#classifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn import svm

from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv(r"/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
df_ballbyball = pd.read_csv(r"/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")

In [ ]:
print(df1.shape)
df1.head()

In [ ]:
df1.info()

In [ ]:
df1.columns

In [ ]:
print(df_ballbyball.shape)
df_ballbyball.head()

In [ ]:
df_ballbyball.info()

In [ ]:
df_ballbyball.columns

In [ ]:
df1.replace(to_replace=["Delhi Daredevils", "Deccan Chargers", "Rising Pune Supergiants"], value=["Delhi Capitals","Sunrisers Hyderabad", "Rising Pune Supergiant"],inplace=True)
df_ballbyball.replace(to_replace=["Delhi Daredevils", "Deccan Chargers", "Rising Pune Supergiants"], value=["Delhi Capitals","Sunrisers Hyderabad", "Rising Pune Supergiant"], inplace= True)

In [ ]:
df = pd.merge(df1, df_ballbyball, left_on='id', right_on='id',how='outer')
#df = df.drop(['umpire1','umpire2'], axis = 1) 
df.head()

In [ ]:
df.shape

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = pd.DatetimeIndex(df['date']).year

In [ ]:
x=['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions', 'Rising Pune Supergiant', 'Royal Challengers Bangalore', 
    'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
    'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
    'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants', 'Delhi Capitals']

y = ['SRH','MI','GL','RPS','RCB','KKR','DC','KXIP','CSK','RR','SRH','KTK','PW','RPS','DC']

df.replace(x,y,inplace = True)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df.drop(['method'], axis = 1, inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
df.apply(lambda x: x.dtype)

In [ ]:
df.columns

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

In [ ]:
round((df.apply(lambda x:x.isnull().sum())/len(df))*100,2)

In [ ]:
#Checking for percentage of missing values in each columns
(df.isnull().sum()/len(df))*100

In [ ]:
print('Unique Values for Each Feature: \n')
for i in df.columns:
    print(i, ':',df[i].nunique()) 

In [ ]:
# find the unique values from categorical features
for col in df.select_dtypes(include='object').columns:
    print(col)
    print(df[col].unique())

In [ ]:
df.shape

In [ ]:
df.dropna(how = 'any').shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.loc[df.duplicated(keep = 'last'), :]

In [ ]:
df.loc[df.duplicated(keep = False), :]

In [ ]:
df.drop_duplicates(keep = 'first').shape

In [ ]:
df.drop_duplicates(keep = 'last').shape

In [ ]:
df.drop_duplicates(keep = False).shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.columns.to_series().groupby(df.dtypes).groups

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
total_miss = df.isnull().sum()
perc_miss = total_miss/df.isnull().count()*100

missing_data = pd.DataFrame({'Total missing':total_miss,'% missing':perc_miss})

missing_data.sort_values(by='Total missing',ascending=False).head(3)

In [ ]:
#Checking for percentage of missing values in each columns
(df.isnull().sum()/len(df))*100

In [ ]:
categorical_data = df.select_dtypes(exclude=np.number) # we will exclude data with numeric features
categorical_col = categorical_data.columns                          # we will store the categorical features in a variable

print("Categorical Features:")
print(categorical_data.head())
print("===="*20)

In [ ]:
numerical_data = df.select_dtypes(include=np.number) # select_dtypes selects data with numeric features
numerical_col = numerical_data.columns 

print("Numeric Features:")
print(numerical_data.head())
print("===="*20)

In [ ]:
categorical_features = [feature for feature in df.columns if ((df[feature].dtypes=='O') & (feature not in ['deposit']))]
categorical_features

In [ ]:
### numerical 
numerical_cols = list(df.select_dtypes(exclude=['object']))
numerical_cols

In [ ]:
### categorical
categorical_cols = list(df.select_dtypes(include=['object']))
categorical_cols

In [ ]:
# list of numerical variables
numerical_features = [feature for feature in df.columns if ((df[feature].dtypes != 'O') & (feature not in ['y']))]
print('Number of numerical variables: ', len(numerical_features))

In [ ]:
#Discrete Numerical Features
discrete_feature=[feature for feature in numerical_features if len(df[feature].unique())<25]
print("Discrete Variables Count: {}".format(len(discrete_feature)))

In [ ]:
#Continuous Numerical Features
continuous_features=[feature for feature in numerical_features if feature not in discrete_feature+['deposit']]
print("Continuous feature Count: {}".format(len(continuous_features)))

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
df.hist(figsize=(20,20))
plt.show()

In [ ]:
sns.pairplot(df)

In [ ]:
matrix = df.corr() 
f, ax = plt.subplots(figsize=(25, 12)) 
sns.heatmap(matrix, vmax=.8, square=True, cmap="RdYlGn",annot = True);

In [ ]:
def bar_plot(variable):
    var = df[variable]
    varValue = var.value_counts()
    plt.figure(figsize=(15,3))
    plt.bar(varValue.index, varValue,color=['#00008b','#00e5ee','#cd1076', '#008080','#cd5555','red','blue'])
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.xticks(rotation = 90)
    
    plt.show()
    print("{}: \n {}".format(variable,varValue))

In [ ]:
categorical_cols = ['city', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result', 'eliminator', 
                    'umpire1', 'umpire2', 'dismissal_kind', 'extras_type', 'batting_team', 'bowling_team']
for c in categorical_cols:
    bar_plot(c)

# Most Wins In IPL

In [ ]:
temp = pd.DataFrame({'Winner': df['winner']})
count_wins = temp.value_counts()
print(count_wins)

In [ ]:
temp = pd.DataFrame({'Winner': df['winner']})
count_wins = temp.value_counts()
labels = [X[0] for X in count_wins.keys()]
labels

In [ ]:
bar,ax = plt.subplots(figsize = (20,12))
ax = plt.pie(x = count_wins, autopct = "%.1f%%", labels = labels)
plt.title('Most Wins In IPL', fontsize = 17)
plt.show()

# Most Wins In Eliminator

In [ ]:
sns.countplot(df['winner'][df['eliminator'] == 'Y'], data = df)
plt.title("Most Wins In Eliminator", fontsize = 17)
plt.xticks(rotation = 90)
plt.show()

# Toss Decision

In [ ]:
teams = df['toss_winner'].unique()

decision_making = pd.DataFrame([], columns = ['Toss Winner', 'Decision', 'Times'])

for id, element in enumerate(teams):
    temp_bat = df[(df['toss_winner'] == element) & (df['toss_decision'] == 'bat')]
    temp_field = df[(df['toss_winner'] == element) & (df['toss_decision'] == 'field')]
    
    #Append to decision_making
    decision_making = decision_making.append({'Toss Winner': element, 'Decision' : 'bat', 
                                              'Times' : temp_bat['toss_winner'].count()}, ignore_index = True)
    
    decision_making = decision_making.append({'Toss Winner': element, 'Decision' : 'field', 
                                              'Times' : temp_field['toss_winner'].count()}, ignore_index = True)
decision_making

In [ ]:
sns.catplot(x =  'Toss Winner', y = 'Times', hue = 'Decision', data = decision_making, kind = 'bar', height = 5, aspect = 2)
plt.title("Toss Decision of Teams")
plt.xlabel("IPL Teams")
plt.ylabel("Toss Decision")

# Famous Venue

In [ ]:
sns.barplot(x = df['venue'].value_counts().head(8).values, y = df['venue'].value_counts().head(8).index, data = df)
plt.title("Famous Venue")
plt.xlabel("Venue Count")
plt.ylabel("Venue")

# Top 5 Umpire_1

In [ ]:
sns.barplot(x = df['umpire1'].value_counts().head(8).values, y = df['umpire1'].value_counts().head(8).index, data = df)
plt.title("Umpire_1")
plt.xlabel("Match Count")
plt.ylabel("Top 5 Umpire_1")

# Top 5 Umpire 2

In [ ]:
sns.barplot(x = df['umpire2'].value_counts().head(10).values, y = df['umpire2'].value_counts().head(10).index, data = df)
plt.title("Umpire 2")
plt.xlabel("Match Count")
plt.ylabel("Top 5 Umpire 2")

**Before Combining(Concat) the both csv files, "Fielder" is the Target Variable in Ball-By-Ball and "Method" is the Target Variable in Matches.**

**When combining(concat) the both csv files, Here the "Fielder" is the Traget Variable.**

# Univariate Analysis

In [ ]:
df['city'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'city',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['city'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['venue'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'venue',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['venue'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['neutral_venue'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = 'neutral_venue',data = df)

In [ ]:
plt.figure(figsize=(10,5))
df['neutral_venue'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['team1'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'team1',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['team1'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['team2'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'team2',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['team2'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['toss_winner'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'toss_winner',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['toss_winner'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['toss_decision'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'toss_decision',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['toss_decision'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['winner'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'winner',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['winner'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['result'].value_counts()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(y = 'result',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['result'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['eliminator'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = 'eliminator',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['eliminator'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['umpire1'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(y = 'umpire1',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['umpire1'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['umpire2'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(y = 'umpire2',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['umpire2'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['inning'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = 'inning',data = df)

In [ ]:
plt.figure(figsize=(10,10))
df['inning'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['over'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(y = 'over',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['over'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['ball'].value_counts()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y = 'ball',data = df)

In [ ]:
plt.figure(figsize=(15,10))
df['ball'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['batsman'].value_counts()

In [ ]:
df['non_striker'].value_counts()

In [ ]:
df['bowler'].value_counts()

In [ ]:
df['batsman_runs'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['batsman_runs'])

In [ ]:
plt.figure(figsize=(10,10))
df['batsman_runs'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['extra_runs'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['extra_runs'])

In [ ]:
plt.figure(figsize=(10,10))
df['extra_runs'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df["total_runs"].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['total_runs'])

In [ ]:
plt.figure(figsize=(10,10))
df['total_runs'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['non_boundary'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['non_boundary'])

In [ ]:
plt.figure(figsize=(10,10))
df['non_boundary'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['is_wicket'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['is_wicket'])

In [ ]:
plt.figure(figsize=(10,10))
df['is_wicket'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['dismissal_kind'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['dismissal_kind'])

In [ ]:
plt.figure(figsize=(15,10))
df['dismissal_kind'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['player_dismissed'].value_counts()

In [ ]:
df['fielder'].value_counts()

In [ ]:
df['extras_type'].value_counts()

In [ ]:
df['batting_team'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['batting_team'])

In [ ]:
plt.figure(figsize=(10,10))
df['batting_team'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['bowling_team'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(y = df['bowling_team'])

In [ ]:
plt.figure(figsize=(10,10))
df['bowling_team'].value_counts().plot.pie(autopct="%0.2f%%")

In [ ]:
df['city'] = df['city'].map({'Mumbai': 0, 'Kolkata': 1, 'Delhi': 2, 'Bangalore': 3, 'Hyderabad': 4,'Chennai': 5, 
                             'Chandigarh': 6, 'Jaipur': 7, 'Pune': 8, 'Abu Dhabi': 9, 'Dubai': 10, 'Durban': 11, 
                             'Bengaluru': 12, 'Visakhapatnam': 13, 'Sharjah': 14, 'Ahmedabad': 15, 'Centurion': 16, 
                             'Rajkot': 17, 'Dharamsala': 18, 'Indore': 19, 'Johannesburg': 20, 'Port Elizabeth': 21, 
                             'Cape Town': 22, 'Cuttack': 23, 'Ranchi': 24, 'Raipur': 25, 'Kochi': 26, 'Kanpur': 27, 
                             'Kimberley': 28, 'Nagpur': 29, 'East London':  30, 'Bloemfontein': 31})
df.head()

In [ ]:
df['player_of_match'] = df['player_of_match'].map({'BB McCullum': 1, 'MEK Hussey': 2, 'MF Maharoof': 3, 'MV Boucher': 4, 
                                                   'DJ Hussey': 5, 'SR Watson': 6, 'V Sehwag': 7, 'ML Hayden': 8, 
                                                   'YK Pathan': 9, 'KC Sangakkara': 10, 'JDP Oram': 11, 'AC Gilchrist': 12, 
                                                   'SM Katich': 13, 'MS Dhoni': 14, 'ST Jayasuriya': 15, 'GD McGrath': 16, 
                                                   'SE Marsh': 17, 'SA Asnodkar': 18, 'R Vinay Kumar': 19, 'IK Pathan': 20, 
                                                   'SM Pollock': 21, 'Sohail Tanvir': 22, 'S Sreesanth': 23, 'A Nehra': 24, 
                                                   'SC Ganguly': 25, 'CRD Fernando': 26, 'L Balaji': 27, 
                                                   'Shoaib Akhtar': 28, 'A Mishra': 29, 'DPMD Jayawardene': 30, 
                                                   'GC Smith': 31, 'DJ Bravo': 32, 'M Ntini': 33, 'SP Goswami': 34, 
                                                   'A Kumble': 35, 'KD Karthik': 36, 'JA Morkel': 37, 'P Kumar': 38, 
                                                   'Umar Gul': 39, 'SK Raina': 40, 'SR Tendulkar': 41, 'R Dravid': 42, 
                                                   'DL Vettori': 43, 'RP Singh': 44, 'M Muralitharan': 45, 'CH Gayle': 46, 
                                                   'AB de Villiers': 47, 'RS Bopara': 48, 'PP Ojha': 49, 'TM Dilshan': 50, 
                                                   'HH Gibbs': 51, 'DP Nannes': 52, 'JP Duminy': 53, 'Yuvraj Singh': 54, 
                                                   'SB Jakati': 55, 'JH Kallis': 56, 'G Gambhir': 57, 'RG Sharma': 58, 
                                                   'A Singh': 59, 'S Badrinath': 60, 'DR Smith': 61, 'LRPL Taylor': 62, 
                                                   'Harbhajan Singh': 63, 'R Bhatia': 64, 'SK Warne': 65, 'B Lee': 66, 
                                                   'BJ Hodge': 67, 'LR Shukla': 68, 'MK Pandey': 69, 'AD Mathews': 70, 
                                                   'MK Tiwary': 71, 'WPUJC Vaas': 72, 'A Symonds': 73, 
                                                   'AA Jhunjhunwala': 74, 'J Theron': 75, 'RV Uthappa': 76, 'AC Voges': 77, 
                                                   'KM Jadhav': 78, 'NV Ojha': 79, 'DA Warner': 80, 'SL Malinga': 81, 
                                                   'M Vijay': 82, 'KP Pietersen': 83, 'AT Rayudu': 84, 'PD Collingwood': 85, 
                                                   'MJ Lumb': 86, 'TL Suman': 87, 'RJ Harris': 88, 'PP Chawla': 89, 
                                                   'Harmeet Singh': 90, 'KA Pollard': 91, 'R Ashwin': 92, 'R McLaren': 93, 
                                                   'JD Unadkat': 94, 'M Kartik': 95, 'DE Bollinger': 96, 'S Anirudha': 97, 
                                                   'SK Trivedi': 98, 'SB Wagh': 99, 'PC Valthaty': 100, 'MD Mishra': 101, 
                                                   'DW Steyn': 102, 'S Sohal': 103, 'MM Patel': 104, 'V Kohli': 105, 
                                                   'I Sharma': 106, 'J Botha': 107, 'Iqbal Abdulla': 108, 
                                                   'P Parameswaran': 109, 'R Sharma': 110, 'MR Marsh': 111, 'BA Bhatt': 112, 
                                                   'S Aravind': 113, 'WP Saha': 114, 'S Dhawan': 115, 'JEC Franklin': 116,
                                                   'RE Levi': 117, 'SPD Smith': 118, 'AM Rahane': 119, 'RA Jadeja': 120, 
                                                   'MN Samuels': 121,'M Morkel': 122, 'F du Plessis': 123, 
                                                   'AD Mascarenhas': 124, 'Shakib Al Hasan': 125, 'JD Ryder': 126, 
                                                   'SP Narine': 127, 'S Nadeem': 128, 'KMDN Kulasekara': 129, 
                                                   'CL White': 130, 'Mandeep Singh': 131, 'P Negi': 132, 
                                                   'Azhar Mahmood': 133, 'BW Hilfenhaus': 134, 'A Chandila': 135, 
                                                   'UT Yadav': 136, 'MS Bisla': 137,'M Vohra': 138, 'GH Vihari': 139, 
                                                   'AJ Finch': 140, 'JP Faulkner': 141, 'MS Gony': 142, 'DA Miller': 143, 
                                                   'SV Samson': 144, 'DJG Sammy': 145, 'MG Johnson': 146, 'KK Cooper': 147, 
                                                   'PA Patel':148, 'AP Tare': 149, 'LJ Wright': 150, 'YS Chahal': 151, 
                                                   'GJ Maxwell': 152, 'CA Lynn': 153, 'MM Sharma': 154, 'PV Tambe': 155, 
                                                   'Sandeep Sharma': 156, 'B Kumar': 157, 'CJ Anderson': 158, 
                                                   'KK Nair': 159, 'AR Patel': 160, 'LMP Simmons': 161, 'DJ Hooda': 162, 
                                                   'GJ Bailey': 163, 'AD Russell': 164, 'MA Agarwal': 165, 'SS Iyer': 166, 
                                                   'MA Starc': 167, 'VR Aaron': 168, 'TA Boult': 169, 
                                                   'NM Coulter-Nile': 170, 'EJG Morgan': 171, 'HH Pandya': 172, 
                                                   'MC Henriques': 173, 'Z Khan': 174, 'MJ McClenaghan': 175, 
                                                   'Q de Kock': 176, 'Mustafizur Rahman': 177, 'SA Yadav': 178, 
                                                   'AB Dinda': 179, 'CH Morris': 180, 'CR Brathwaite': 181, 'RR Pant': 182, 
                                                   'MP Stoinis': 183, 'A Zampa': 184, 'KH Pandya': 185, 'HM Amla': 186, 
                                                   'BCJ Cutting': 187, 'Rashid Khan': 188, 'N Rana': 189, 'JJ Bumrah': 190, 
                                                   'AJ Tye': 191, 'BA Stokes': 191, 'KS Williamson': 192, 'JC Buttler': 193, 
                                                   'LH Ferguson': 194, 'Mohammed Shami': 195, 'RA Tripathi': 196, 
                                                   'Mohammed Siraj': 197, 'HV Patel': 198, 'Washington Sundar': 199, 
                                                   'KV Sharma': 200, 'KL Rahul': 201, 'SW Billings': 202, 'JJ Roy': 203, 
                                                   'B Stanlake': 204, 'JC Archer': 205, 'AS Rajpoot': 206, 
                                                   'TG Southee': 207, 'Mujeeb Ur Rahman': 208, 'Ishan Kishan': 209, 
                                                   'Kuldeep Yadav': 210, 'S Gopal': 211, 'L Ngidi': 212, 'PP Shaw': 213, 
                                                   'JM Bairstow': 215, 'SM Curran': 216, 'AS Joseph': 217, 'K Rabada': 218, 
                                                   'HF Gurney': 219, 'DL Chahar': 220, 'Imran Tahir': 221, 'KMA Paul': 222, 
                                                   'KK Ahmed': 223, 'Shubman Gill': 224, 'SO Hetmyer': 225, 'CV Varun': 226, 
                                                   'CJ Jordan': 227, 'Shivam Mavi': 228, 'A Nortje': 229, 'RD Gaikwad': 230, 
                                                   'R Tewatia': 231, 'PK Garg': 232, 'PJ Cummins': 233})
df.head()

In [ ]:
df['venue'] = df['venue'].map({'M Chinnaswamy Stadium': 0, 'Punjab Cricket Association Stadium, Mohali': 1, 
                               'Feroz Shah Kotla': 2, 'Wankhede Stadium': 3, 'Eden Gardens': 4, 
                               'Sawai Mansingh Stadium': 5, 'Rajiv Gandhi International Stadium, Uppal': 6, 
                               'MA Chidambaram Stadium, Chepauk': 7, 'Dr DY Patil Sports Academy': 8, 'Newlands': 9, 
                               "St George's Park": 10, 'Kingsmead': 11, 'SuperSport Park': 12, 'Buffalo Park': 13, 
                               'New Wanderers Stadium': 14, 'De Beers Diamond Oval': 15, 'OUTsurance Oval': 16, 
                               'Brabourne Stadium': 17, 'Sardar Patel Stadium, Motera': 18, 'Barabati Stadium': 19, 
                               'Vidarbha Cricket Association Stadium, Jamtha': 20, 
                               'Himachal Pradesh Cricket Association Stadium': 21, 'Nehru Stadium': 22, 
                               'Holkar Cricket Stadium': 23, 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 24, 
                               'Subrata Roy Sahara Stadium': 25, 'Shaheed Veer Narayan Singh International Stadium': 26, 
                               'JSCA International Stadium Complex': 27, 'Sheikh Zayed Stadium': 28, 
                               'Sharjah Cricket Stadium': 29, 'Dubai International Cricket Stadium': 30, 
                               'Maharashtra Cricket Association Stadium': 31, 
                               'Punjab Cricket Association IS Bindra Stadium, Mohali': 32, 
                               'Saurashtra Cricket Association Stadium': 33, 'Green Park': 34, 'M.Chinnaswamy Stadium': 35})
df.head()

In [ ]:
df['team1'] = df['team1'].map({'RCB': 0, 'KXIP': 1, 'DC': 2, 'MI': 3, 'KKR': 4, 'RR': 5, 'SRH': 6, 'CSK': 7, 'KTK': 8, 'PW': 9, 'GL': 10, 'RPS': 11})
df.head()

In [ ]:
df['team2'] = df['team2'].map({'KKR': 0, 'CSK': 1, 'RR': 2, 'RCB': 3, 'SRH': 4, 'KXIP': 5, 'DC': 6, 'MI': 7, 'KTK': 8, 'PW': 9, 'RPS': 10, 'GL': 11})
df.head()

In [ ]:
df['toss_winner'] = df['toss_winner'].map({'RCB': 0, 'CSK': 1, 'RR': 2, 'MI': 3, 'SRH': 4, 'KXIP': 5, 'KKR': 6, 'DC': 7, 'KTK': 8, 'PW': 9, 'GL': 10, 
                                           'RPS': 11})
df.head()

In [ ]:
df['toss_decision'] = df['toss_decision'].map({'field': 0, 'bat': 1})
df.head()

In [ ]:
df['winner'] = df['winner'].map({'KKR': 0, 'CSK': 1, 'DC': 2, 'RCB': 3, 'RR': 4, 'KXIP': 5, 'SRH': 6, 'MI': 7, 'PW': 8, 'KTK': 9, 'RPS': 10, 'GL': 11})
df.head()

In [ ]:
df['result'] = df['result'].map({'wickets': 0, 'runs': 1, 'tie': 2})
df.head()

In [ ]:
df['eliminator'] = df['eliminator'].map({'N': 0, 'Y': 1})
df.head()

In [ ]:
df['umpire1'] = df['umpire1'].map({'Asad Rauf': 0, 'MR Benson': 1, 'Aleem Dar': 2, 'SJ Davis': 3, 'BF Bowden': 4, 'IL Howell': 5, 'DJ Harper': 6, 
                                   'RE Koertzen': 7, 'BR Doctrove': 8, 'AV Jayaprakash': 9, 'BG Jerling': 10, 'M Erasmus': 11, 'HDPK Dharmasena': 12, 
                                   'S Asnani': 13, 'GAV Baxter':14, 'SS Hazare': 15, 'K Hariharan': 16, 'SL Shastri': 17, 'SK Tarapore': 18, 'S Ravi': 19, 
                                   'SJA Taufel': 20, 'S Das': 21, 'AM Saheba': 22, 'PR Reiffel': 23, 'JD Cloete': 24, 'AK Chaudhary': 25, 
                                   'VA Kulkarni': 26, 'BNJ Oxenford': 27, 'CK Nandan': 28, 'C Shamshuddin': 29, 'NJ Llong': 30, 'RK Illingworth': 31, 
                                   'RM Deshpande': 32, 'K Srinath': 33, 'SD Fry': 34, 'CB Gaffaney': 35, 'PG Pathak': 36, 'Nitin Menon': 37, 
                                   'K Bharatan': 38, 'AY Dandekar': 39, 'KN Ananthapadmanabhan': 40, 'A Nand Kishore': 41, 'A Deshmukh': 42, 
                                   'YC Barde': 43, 'RJ Tucker': 44, 'VK Sharma': 45, 'UV Gandhe': 46, 'IJ Gould': 47})
df.head()

In [ ]:
df['umpire2'] = df['umpire2'].map({'RE Koertzen': 0, 'SL Shastri': 1, 'GA Pratapkumar': 2, 'DJ Harper': 3, 'K Hariharan': 4, 'RB Tiffin': 5, 
                                   'AM Saheba': 6, 'MR Benson': 7, 'IL Howell': 8, 'AV Jayaprakash': 9, 'I Shivram': 10, 'BR Doctrove': 11, 
                                   'BG Jerling': 12, 'SJ Davis': 13, 'SD Ranade': 14, 'SJA Taufel': 15, 'M Erasmus': 16, 'TH Wijewardene': 17, 
                                   'SK Tarapore': 18, 'S Ravi': 19, 'HDPK Dharmasena': 20, 'SS Hazare': 21, 'PR Reiffel': 22, 'AL Hill': 23, 
                                   'RJ Tucker': 24, 'VA Kulkarni': 25, 'JD Cloete': 26, 'BNJ Oxenford': 27, 'S Asnani': 28, 'S Das': 29, 
                                   'C Shamshuddin': 30, 'AK Chaudhary': 31, 'K Srinath': 32, 'Subroto Das': 33, 'CK Nandan': 34, 'NJ Llong': 35, 
                                   'RK Illingworth': 36, 'PG Pathak': 37, 'CB Gaffaney': 38, 'K Srinivasan': 39, 'SD Fry': 40, 'VK Sharma': 41, 
                                   'A Nand Kishore': 42, 'Nitin Menon': 43, 'A Deshmukh': 44, 'YC Barde': 45, 'IJ Gould': 46})
df.head()

In [ ]:
df['batsman'] = df['batsman'].map({'RT Ponting': 0, 'BB McCullum': 1, 'DJ Hussey': 2, 'Mohammad Hafeez': 3, 'SC Ganguly': 4, 
                                   'CL White': 5, 'MV Boucher': 6, 'B Akhil': 7, 'AA Noffke': 8, 'P Kumar': 9, 'Z Khan': 10, 
                                   'SB Joshi': 11, 'W Jaffer': 12, 'JH Kallis': 13, 'R Dravid': 14, 'V Kohli': 15, 'K Goel': 16, 
                                   'MEK Hussey': 17, 'JR Hopes': 18, 'SK Raina': 19, 'JDP Oram': 20, 'S Badrinath': 21, 'PA Patel': 22, 
                                   'ML Hayden': 23, 'MS Dhoni': 24, 'KC Sangakkara': 25, 'Yuvraj Singh': 26, 'SM Katich': 27, 
                                   'IK Pathan': 28, 'T Kohli': 29, 'YK Pathan': 30, 'S Dhawan': 31, 'G Gambhir': 32, 'SK Warne': 33, 
                                   'D Salunkhe': 34, 'SK Trivedi': 35, 'V Sehwag': 36, 'SR Watson': 37, 'M Kaif': 38, 'DS Lehmann': 39, 
                                   'RA Jadeja': 40, 'M Rawat': 41, 'L Ronchi': 42, 'ST Jayasuriya': 43, 'DJ Thornely': 44, 
                                   'RV Uthappa': 45, 'PR Shah': 46, 'AM Nayar': 47, 'SM Pollock': 48, 'Harbhajan Singh': 49, 
                                   'S Chanderpaul': 50, 'LRPL Taylor': 51, 'WP Saha': 52, 'AS Yadav': 53, 'A Symonds': 54, 
                                   'SB Bangar': 55, 'WPUJC Vaas': 56, 'RP Singh': 57, 'AC Gilchrist': 58, 'Y Venugopal Rao': 59, 
                                   'VVS Laxman': 60, 'RG Sharma': 61, 'SB Styris': 62, 'LR Shukla': 63, 'PP Chawla': 64, 
                                   'Kamran Akmal': 65, 'DPMD Jayawardene': 66, 'S Sohal': 67, 'B Lee': 68, 'WA Mota': 69, 
                                   'Shahid Afridi': 70, 'DJ Bravo': 71, 'A Nehra': 72, 'MA Khote': 73, 'GC Smith': 74, 
                                   'Pankaj Singh': 75, 'S Sreesanth': 76, 'VRV Singh': 77, 'SS Tiwary': 78, 'RR Sarwan': 79, 
                                   'DS Kulkarni': 80, 'R Vinay Kumar': 81, 'M Kartik': 82, 'I Sharma': 83, 'AB Agarkar': 84, 
                                   'AM Rahane': 85, 'TM Srivastava': 86, 'Shoaib Malik': 87, 'MK Tiwary': 88, 'KD Karthik': 89, 
                                   'R Bhatia': 90, 'MF Maharoof':91, 'VY Mahesh': 92, 'B Chipli': 93, 'DW Steyn': 94, 
                                   'DB Das': 95, 'MK Pandey': 96, 'HH Gibbs': 97, 'DNT Zoysa': 98, 'D Kalyankrishna': 99, 
                                   'SE Marsh': 100, 'SA Asnodkar': 101, 'BJ Hodge': 102, 'Sohail Tanvir': 103, 'Salman Butt': 104, 
                                   'Umar Gul': 105, 'AB Dinda': 106, 'AB de Villiers': 107, 'SP Fleming': 108, 'S Vidyut': 109, 
                                   'JA Morkel': 110, 'LPC Silva': 111, 'DB Ravi Teja': 112, 'Misbah-ul-Haq': 113, 'PJ Sangwan': 114,
                                   'YV Takawale': 115, 'RR Raje': 116, 'Mohammad Asif': 117, 'GD McGrath': 118, 'Joginder Sharma': 119, 
                                   'MS Gony': 120, 'M Muralitharan': 121, 'M Ntini': 122, 'DT Patil': 123, 'A Kumble': 124, 
                                   'S Anirudha': 125, 'MM Patel': 126, 'CK Kapugedera': 127, 'A Chopra': 128, 'T Taibu': 129, 
                                   'J Arunkumar': 130, 'PP Ojha': 131, 'SP Goswami': 132, 'SR Tendulkar': 133, 'U Kaul': 134, 
                                   'Niraj Patel': 135, 'AD Mascarenhas': 136, 'TM Dilshan': 137, 'A Mishra': 138, 'LA Pomersbach': 139,
                                   'Iqbal Abdulla': 140, 'Younis Khan': 141, 'PM Sarvesh Kumar': 142, 'DP Vijaykumar': 143, 
                                   'Shoaib Akhtar': 144, 'Abdur Razzak': 145, 'H Das': 146, 'DR Smith': 147, 'SD Chitnis': 148, 
                                   'CRD Fernando': 149, 'VS Yeligati': 150, 'L Balaji': 151, 'A Mukund': 152, 'RR Powar': 153, 
                                   'JP Duminy': 154, 'A Flintoff': 155, 'T Thushara': 156, 'KP Pietersen': 157, 'JD Ryder': 158, 
                                   'T Henderson': 159, 'Kamran Khan': 160, 'RS Bopara': 161, 'MC Henriques': 162, 'CH Gayle': 163, 
                                   'R Bishnoi': 164, 'KV Sharma': 165, 'FH Edwards': 166, 'AS Raut': 167, 'PC Valthaty': 168, 
                                   'RJ Quiney': 169, 'Yashpal Singh': 170, 'M Manhas': 171, 'AA Bilakhia': 172, 'AN Ghosh': 173, 
                                   'BAW Mendis': 174, 'DL Vettori': 175, 'MN van Wyk': 176, 'RE van der Merwe': 177, 'TL Suman': 178, 
                                   'Shoaib Ahmed': 179, 'GR Napier': 180, 'KP Appanna': 181, 'LA Carseldine': 182, 'NV Ojha': 183, 
                                   'SM Harwood': 184, 'M Vijay': 185, 'DA Warner': 186, 'SB Jakati': 187, 'RJ Harris': 188, 
                                   'D du Preez': 189, 'M Morkel': 190, 'AD Mathews': 191, 'SL Malinga': 192, 'C Nanda': 193, 
                                   'J Botha': 194, 'Mashrafe Mortaza': 195, 'A Singh': 196, 'GJ Bailey': 197, 'AB McDonald': 198, 
                                   'Y Nagar': 199, 'SS Shaikh': 200, 'R Ashwin': 201, 'Mohammad Ashraful': 202, 'Jaskaran Singh': 203, 
                                   'Anirudh Singh': 204, 'CA Pujara': 205, 'OA Shah': 206, 'AP Tare': 207, 'AT Rayudu': 208, 
                                   'R Sathish': 209, 'R McLaren': 210, 'AA Jhunjhunwala': 211, 'P Dogra': 212, 'A Uniyal': 213, 
                                   'MS Bisla': 214, 'YA Abdulla': 215, 'EJG Morgan': 216, 'JM Kemp': 217, 'S Tyagi': 218, 
                                   'RS Gavaskar': 219, 'SE Bond': 220, 'S Ladda': 221, 'DP Nannes': 222, 'KA Pollard': 225, 
                                   'DR Martyn': 226, 'S Narwal': 227,  'MJ Lumb': 228, 'Bipul Sharma': 229, 'AB Barath': 230, 
                                   'FY Fazal': 231, 'AC Voges': 232, 'MD Mishra': 233, 'UT Yadav': 234, 'J Theron': 235, 
                                   'SJ Srivastava': 236, 'R Sharma': 237, 'Mandeep Singh': 238, 'KM Jadhav': 239, 'SW Tait': 240, 
                                   'KB Arun Karthik': 241, 'KAJ Roach': 242, 'PD Collingwood': 243, 'CK Langeveldt': 244, 
                                   'VS Malik': 245, 'A Mithun': 246, 'AP Dole': 247, 'AN Ahmed': 248, 'RS Sodhi': 249, 
                                   'DE Bollinger':250, 'S Sriram': 251, 'B Sumanth': 252, 'C Madan': 253, 'AG Paunikar': 254, 
                                   'MR Marsh': 255, 'AJ Finch': 256, 'STR Binny': 257, 'Harmeet Singh': 258, 'IR Jaggi': 259, 
                                   'DT Christian': 260, 'MA Agarwal': 261, 'RV Gomez': 262, 'AUK Pathan': 263, 'UBT Chand': 264, 
                                   'DJ Jacobs': 265, 'NJ Rimmington': 266, 'Sunny Singh': 267, 'AL Menaria': 268, 'WD Parnell': 269, 
                                   'JJ van der Wath': 270, 'R Ninan': 271, 'S Aravind': 272, 'MS Wade': 273, 'TD Paine': 274, 
                                   'JEC Franklin': 275, 'SB Wagh': 276, 'AC Thomas': 277, 'Shakib Al Hasan': 278, 'DH Yagnik': 279, 
                                   'S Randiv': 280, 'BJ Haddin': 281, 'NLTC Perera': 282, 'NL McCullum': 283, 'JE Taylor': 284, 
                                   'J Syed Mohammad': 285, 'RN ten Doeschate': 286, 'TR Birt': 287, 'AG Murtaza': 288, 
                                   'Harpreet Singh': 289, 'M Klinger': 290, 'AC Blizzard': 291, 'I Malhotra': 292, 'L Ablish': 293, 
                                   'CA Ingram': 294, 'S Nadeem': 295, 'P Parameswaran': 296, 'CJ Ferguson': 297, 'VR Aaron': 298, 
                                   'AA Chavan': 299, 'ND Doshi': 300, 'B Kumar': 301, 'Y Gnaneswara Rao': 302, 'S Rana': 303, 
                                   'BA Bhatt': 304, 'F du Plessis': 305, 'RE Levi': 306, 'GJ Maxwell': 307, 'SPD Smith': 308, 
                                   'SA Yadav': 309, 'MN Samuels': 310, 'KK Cooper': 311, 'JP Faulkner': 312, 'HV Patel': 313, 
                                   'DAJ Bracewell': 314, 'DJ Harris': 315, 'Ankit Sharma': 316, 'SP Narine': 317, 'GB Hogg': 318, 
                                   'RR Bhatkal': 319, 'CJ McKay': 320, 'N Saini': 321, 'DA Miller': 322, 'Azhar Mahmood': 323, 
                                   'P Negi': 324, 'RJ Peterson': 325, 'KMDN Kulasekara': 326, 'A Ashish Reddy': 327, 
                                   'V Pratap Singh': 328, 'BB Samantray': 329, 'MJ Clarke': 330, 'Gurkeerat Singh': 331, 
                                   'AP Majumdar': 332, 'PA Reddy': 333, 'K Upadhyay': 334, 'P Awana': 335, 'AD Russell': 336, 
                                   'A Chandila': 337, 'CA Lynn': 338, 'Sunny Gupta': 339, 'MC Juneja': 340, 'KK Nair': 341, 
                                   'GH Vihari': 342, 'R Shukla': 343, 'MDKJ Perera': 344, 'AS Rajpoot': 345, 'B Laughlin': 346, 
                                   'M Vohra': 347, 'JD Unadkat': 348, 'Mohammed Shami': 349, 'BMAJ Mendis': 350, 'R Rampaul': 351, 
                                   'CH Morris': 352, 'SV Samson': 353, 'SMSM Senanayake': 354, 'BJ Rohrer': 355, 'KL Rahul': 356, 
                                   'Q de Kock': 357, 'R Dhawan': 358, 'MG Johnson': 359, 'LJ Wright': 360, 'IC Pandey': 361, 
                                   'CM Gautam': 362, 'X Thalaivan Sargunam': 363, 'DJG Sammy': 364, 'KW Richardson': 365, 
                                   'MM Sharma': 366, 'UA Birla': 367, 'Parvez Rasool': 368, 'Sandeep Sharma': 369, 'S Kaul': 370, 
                                   'Sachin Baby': 371, 'PV Tambe': 372, 'NM Coulter-Nile': 373, 'CJ Anderson': 374, 
                                   'NJ Maddinson': 375, 'AR Patel': 376, 'JJ Bumrah': 377, 'JDS Neesham': 378, 'TG Southee': 379, 
                                   'MA Starc': 380, 'BR Dunk': 381, 'RR Rossouw': 382, 'Shivam Sharma': 383, 'YS Chahal': 384, 
                                   'LMP Simmons': 385, 'VH Zol': 386, 'Imran Tahir': 387, 'BCJ Cutting': 388, 'BE Hendricks': 389, 
                                   'S Gopal': 390, 'M de Lange': 391, 'R Tewatia': 392, 'JO Holder': 393, 'Karanveer Singh': 394, 
                                   'SS Iyer': 395, 'DJ Hooda': 396, 'Anureet Singh': 397, 'KS Williamson': 398, 'SA Abbott': 399, 
                                   'J Suchith': 400, 'RG More': 401, 'D Wiese': 402, 'HH Pandya': 403, 'SN Khan': 404, 
                                   'MJ McClenaghan': 405, 'DJ Muthuswami': 406, 'PJ Cummins': 407, 'SN Thakur': 408, 
                                   'JC Buttler': 409, 'CR Brathwaite': 410, 'MP Stoinis': 411, 'Ishan Kishan': 412, 'C Munro': 413, 
                                   'P Sahu': 414, 'KH Pandya': 413, 'AD Nath': 414, 'MJ Guptill': 415, 'KJ Abbott': 416, 'TM Head': 417, 
                                   'M Ashwin': 418, 'NS Naik': 419, 'RR Pant': 420, 'SW Billings': 421, 'KC Cariappa': 422, 
                                   'PSP Handscomb': 423, 'Swapnil Singh': 424, 'J Yadav': 425, 'UT Khawaja': 426, 'HM Amla': 427, 
                                   'F Behardien': 428, 'A Zampa': 429, 'BB Sran': 430, 'N Rana': 431, 'S Kaushik': 432, 
                                   'ER Dwivedi': 433, 'CJ Jordan': 434, 'A Choudhary': 435, 'TS Mills': 436, 'BA Stokes': 437, 
                                   'JJ Roy': 438, 'Vishnu Vinod': 439, 'Basil Thampi': 440, 'CR Woakes': 441, 'RA Tripathi': 442, 
                                   'DL Chahar': 443, 'V Shankar': 444, 'Rashid Khan': 445, 'LH Ferguson': 446, 'RD Chahar': 447, 
                                   'C de Grandhomme': 448, 'AF Milne': 449, 'S Badree': 450, 'Mohammad Nabi': 451, 'AJ Tye': 452, 
                                   'K Rabada': 453, 'Kuldeep Yadav': 454, 'Washington Sundar': 455, 'DM Bravo': 456, 'AR Bawne': 457, 
                                   'SP Jackson': 458, 'Ankit Soni': 459, 'TA Boult': 460, 'E Lewis': 461, 'MA Wood': 462, 
                                   'RK Singh': 463, 'DJM Short': 464, 'K Gowtham': 465, 'TK Curran': 466, 'M Markande': 467, 
                                   'B Stanlake': 468, 'Mujeeb Ur Rahman': 469, 'A Dananjaya': 470, 'Shivam Mavi': 471, 
                                   'Shubman Gill': 472, 'Mohammed Siraj': 473, 'H Klaasen': 474, 'RK Bhui': 475, 'JC Archer': 476, 
                                   'PP Shaw': 477, 'LE Plunkett': 478, 'Mustafizur Rahman': 479, 'AD Hales': 480, 'MK Lomror': 481, 
                                   'DR Shorey': 482, 'MM Ali': 483, 'M Prasidh Krishna': 484, 'P Chopra': 485, 
                                   'JPR Scantlebury-Searles': 486, 'Abhishek Sharma': 487, 'IS Sodhi': 488, 'SO Hetmyer': 489, 
                                   'S Dube': 490, 'NA Saini': 491, 'JM Bairstow': 492, 'Rasikh Salam': 493, 'KMA Paul': 494, 
                                   'N Pooran': 495, 'P Ray Barman': 496, 'S Lamichhane': 497, 'SM Curran': 498, 'GC Viljoen': 499, 
                                   'Avesh Khan': 500, 'HF Gurney': 501, 'SD Lad': 502, 'AS Joseph': 503, 'MJ Santner': 504, 
                                   'R Parag': 505, 'JL Denly': 506, 'LS Livingstone': 507, 'KK Ahmed': 508, 'AJ Turner': 509, 
                                   'SE Rutherford': 510, 'Harpreet Brar': 511, 'Y Prithvi Raj': 512, 'P Simran Singh': 513, 
                                   'JL Pattinson': 514, 'A Nortje': 515, 'T Banton': 516, 'D Padikkal': 515, 'YBK Jaiswal': 516, 
                                   'RD Gaikwad': 517, 'TU Deshpande': 518, 'Abdul Samad': 519, 'PK Garg': 520, 'JR Philippe': 521, 
                                   'Kartik Tyagi': 522, 'CV Varun': 523, 'KL Nagarkoti': 524, 'I Udana': 525, 'Ravi Bishnoi': 526, 
                                   'Shahbaz Ahmed': 527, 'AT Carey': 528, 'N Jagadeesan': 529, 'T Natarajan': 530, 'P Dubey': 531, 
                                   'SS Cottrell': 532, 'Arshdeep Singh': 533, 'DR Sams': 534})
df.head()

In [ ]:
df['non_striker'] = df['non_striker'].map({'BB McCullum': 0, 'RT Ponting': 1, 'DJ Hussey': 2, 'Mohammad Hafeez': 3, 'SC Ganguly': 4,
                                           'MV Boucher': 5, 'CL White': 6, 'B Akhil': 7, 'AA Noffke': 8, 'P Kumar': 9, 'Z Khan': 10,
                                           'SB Joshi': 11, 'JH Kallis': 12, 'W Jaffer': 13, 'R Dravid': 14, 'V Kohli': 15, 
                                           'JR Hopes': 16, 'S Badrinath': 17, 'K Goel': 18, 'SK Raina': 19, 'MEK Hussey': 20, 
                                           'JDP Oram': 21, 'ML Hayden': 22, 'PA Patel': 23, 'MS Dhoni': 24, 'KC Sangakkara': 25, 
                                           'Yuvraj Singh': 26, 'SM Katich': 27, 'IK Pathan': 28, 'YK Pathan': 29, 'T Kohli': 30, 
                                           'G Gambhir': 31, 'S Dhawan': 32, 'D Salunkhe': 33, 'SK Warne': 34, 'SK Trivedi': 35, 
                                           'V Sehwag': 36, 'SR Watson': 37, 'M Kaif': 38, 'DS Lehmann': 39, 'RA Jadeja': 40, 
                                           'M Rawat': 41, 'ST Jayasuriya': 42, 'L Ronchi': 43, 'DJ Thornely': 44, 'RV Uthappa': 45, 
                                           'PR Shah': 46, 'AM Nayar': 47, 'SM Pollock': 48, 'Harbhajan Singh': 49, 'S Chanderpaul': 50, 
                                           'LRPL Taylor': 51, 'WP Saha': 52, 'A Symonds': 53, 'AS Yadav': 54, 'SB Bangar': 55, 
                                           'WPUJC Vaas': 56, 'RP Singh': 57, 'PP Ojha': 58, 'Y Venugopal Rao': 59, 'AC Gilchrist': 60, 
                                           'VVS Laxman': 61, 'SB Styris': 62, 'LR Shukla': 63, 'WA Mota': 64, 'Kamran Akmal': 65, 
                                           'DPMD Jayawardene': 66, 'S Sohal': 67, 'PP Chawla': 68, 'RG Sharma': 69, 
                                           'Shahid Afridi': 70, 'DJ Bravo': 71, 'A Nehra': 72, 'MA Khote': 73, 'Pankaj Singh': 74, 
                                           'GC Smith': 75, 'B Lee': 76, 'S Sreesanth': 77, 'VRV Singh': 78, 'SS Tiwary': 79, 
                                           'RR Sarwan': 80, 'DS Kulkarni': 81, 'R Vinay Kumar': 82, 'M Kartik': 83, 'I Sharma': 84, 
                                           'AB Dinda': 85, 'AB Agarkar': 86, 'AM Rahane':87, 'TM Srivastava': 88, 'Shoaib Malik': 89, 
                                           'MK Tiwary': 90, 'KD Karthik': 91, 'R Bhatia': 92, 'MF Maharoof': 93, 'VY Mahesh': 94, 
                                           'B Chipli': 95, 'DW Steyn': 96, 'DB Das': 97, 'MK Pandey': 98, 'HH Gibbs': 99, 
                                           'DNT Zoysa': 100, 'D Kalyankrishna': 101, 'SE Marsh': 102, 'SA Asnodkar': 103, 
                                           'BJ Hodge': 104, 'Sohail Tanvir': 105, 'Salman Butt': 106, 'Umar Gul': 107, 
                                           'AB de Villiers': 108, 'SP Fleming': 109, 'S Vidyut': 110, 'JA Morkel': 111, 
                                           'Joginder Sharma': 112, 'DB Ravi Teja': 113, 'LPC Silva': 114, 'Misbah-ul-Haq': 115, 
                                           'DP Vijaykumar': 116, 'PJ Sangwan': 117, 'YV Takawale': 118, 'RR Raje': 119, 
                                           'Mohammad Asif': 120, 'GD McGrath': 121, 'MS Gony': 122, 'M Ntini': 123, 'DT Patil': 124, 
                                           'A Kumble': 125, 'S Anirudha': 126, 'MM Patel': 127, 'CK Kapugedera': 128, 'A Chopra': 129, 
                                           'T Taibu': 130, 'J Arunkumar': 131, 'SP Goswami': 132, 'SR Tendulkar': 133, 
                                           'Niraj Patel': 134, 'AD Mascarenhas': 135, 'TM Dilshan': 136, 'A Mishra': 137, 
                                           'LA Pomersbach': 138, 'Iqbal Abdulla': 139, 'Younis Khan': 140, 'PM Sarvesh Kumar': 141, 
                                           'Shoaib Akhtar': 142, 'Abdur Razzak': 143, 'H Das': 144, 'DR Smith': 145, 
                                           'SD Chitnis': 146, 'CRD Fernando': 147, 'A Nel': 148, 'RR Powar': 149, 'JP Duminy': 150,
                                           'A Flintoff': 151, 'KP Pietersen': 152, 'T Henderson': 153, 'Kamran Khan': 154, 
                                           'RS Bopara': 155, 'MC Henriques': 156, 'CH Gayle': 157, 'R Bishnoi': 158, 
                                           'KV Sharma': 159, 'FH Edwards': 160, 'DL Vettori': 161, 'L Balaji': 162, 
                                           'M Muralitharan': 163, 'AS Raut': 164, 'PC Valthaty': 165, 'Yashpal Singh': 166, 
                                           'JD Ryder': 167, 'M Manhas': 168, 'RJ Quiney': 169, 'AA Bilakhia': 170, 'AN Ghosh': 171, 
                                           'BAW Mendis': 172, 'MN van Wyk': 173, 'RE van der Merwe': 174, 'TL Suman': 175, 
                                           'GR Napier': 176, 'KP Appanna': 177, 'LA Carseldine': 178, 'NV Ojha': 179, 
                                           'SM Harwood': 180, 'M Vijay': 181, 'DA Warner': 182, 'SB Jakati': 183, 'S Tyagi': 184, 
                                           'RJ Harris': 185, 'Shoaib Ahmed': 186, 'D du Preez': 187, 'SL Malinga': 189, 
                                           'M Morkel': 190, 'AD Mathews': 191, 'J Botha': 192, 'Mashrafe Mortaza': 193, 
                                           'Jaskaran Singh': 194, 'GJ Bailey': 195, 'AB McDonald': 196, 'Y Nagar': 197, 
                                           'A Singh': 198, 'SS Shaikh': 199, 'R Ashwin': 200, 'T Thushara': 201, 
                                           'Mohammad Ashraful': 202, 'Anirudh Singh': 203, 'CA Pujara': 204, 'OA Shah': 205, 
                                           'AP Tare': 206, 'AT Rayudu': 207, 'R Sathish': 208, 'AA Jhunjhunwala': 209, 
                                           'P Dogra': 210, 'R McLaren': 211, 'MS Bisla': 212, 'YA Abdulla': 213, 
                                           'EJG Morgan': 214, 'JM Kemp': 215, 'A Uniyal': 216, 'RS Gavaskar': 217, 
                                           'SE Bond': 218, 'S Ladda': 219, 'DP Nannes': 220, 'KA Pollard': 221, 'DR Martyn': 222, 
                                           'MJ Lumb': 223, 'Bipul Sharma': 224, 'AB Barath': 225, 'FY Fazal': 226, 'AC Voges': 227,
                                           'MD Mishra': 228, 'UT Yadav': 229, 'J Theron': 230, 'SJ Srivastava': 231, 
                                           'Mandeep Singh': 232, 'KM Jadhav': 233, 'S Narwal': 234, 'SW Tait': 235, 
                                           'KB Arun Karthik': 236, 'KAJ Roach': 237, 'PD Collingwood': 238, 'VS Malik': 239, 
                                           'C Ganapathy': 240, 'R Sharma': 241, 'A Mithun': 242, 'AP Dole': 243, 
                                           'Harmeet Singh': 244, 'AN Ahmed': 245, 'RS Sodhi': 246, 'S Sriram': 247, 
                                           'B Sumanth': 248, 'C Madan': 249, 'AG Paunikar': 250, 'DE Bollinger': 251, 
                                           'MR Marsh': 254, 'AJ Finch': 255, 'STR Binny': 255, 'IR Jaggi': 256, 
                                           'DT Christian': 257, 'MA Agarwal': 258, 'RV Gomez': 259, 'AUK Pathan': 260, 
                                           'DJ Jacobs': 261, 'NJ Rimmington': 262, 'Sunny Singh': 263, 'UBT Chand': 264, 
                                           'AL Menaria': 265, 'WD Parnell': 266, 'R Ninan': 267, 'S Aravind': 268, 
                                           'JJ van der Wath': 269, 'S Nadeem': 270, 'MS Wade': 271, 'TD Paine': 272, 
                                           'AC Thomas': 273, 'JEC Franklin': 274, 'SB Wagh': 275, 'Shakib Al Hasan': 276, 
                                           'DH Yagnik': 277, 'S Randiv': 278, 'BJ Haddin': 279, 'NLTC Perera': 280, 
                                           'NL McCullum': 281, 'JE Taylor': 282, 'J Syed Mohammad': 283, 'RN ten Doeschate': 284, 
                                           'TR Birt': 285, 'AG Murtaza': 286, 'Harpreet Singh': 287, 'M Klinger': 289, 
                                           'AC Blizzard': 290, 'I Malhotra': 291, 'L Ablish': 292, 'P Parameswaran': 293, 
                                           'CJ Ferguson': 294, 'CA Ingram': 295, 'VR Aaron': 296, 'AA Chavan': 297, 
                                           'ND Doshi': 298, 'B Kumar': 299, 'CK Langeveldt': 300, 'Y Gnaneswara Rao': 301, 
                                           'S Rana': 302, 'BA Bhatt':303, 'F du Plessis': 304, 'RE Levi': 305, 
                                           'GJ Maxwell' : 306,'SPD Smith': 307, 'SA Yadav': 308, 'MN Samuels': 309, 
                                           'JP Faulkner': 310, 'DAJ Bracewell': 311, 'DJ Harris': 312, 'Ankit Sharma': 313, 
                                           'TP Sudhindra': 314, 'KK Cooper': 315, 'HV Patel': 316, 'GB Hogg': 317, 
                                           'RR Bhatkal': 318, 'CJ McKay': 319, 'N Saini': 320, 'DA Miller': 321, 
                                           'Azhar Mahmood': 322, 'P Negi': 323, 'RJ Peterson': 324, 'P Awana': 325, 
                                           'KMDN Kulasekara': 326, 'A Ashish Reddy': 327, 'BB Samantray': 328, 'MJ Clarke': 329, 
                                           'Gurkeerat Singh': 330, 'AP Majumdar': 331, 'PA Reddy': 332, 'K Upadhyay': 333, 
                                           'SP Narine': 334, 'AD Russell': 335, 'CA Lynn': 336, 'Sunny Gupta': 337, 
                                           'MC Juneja': 338, 'KK Nair': 340, 'GH Vihari': 341, 'MDKJ Perera': 342, 
                                           'AS Rajpoot': 343, 'B Laughlin': 344, 'M Vohra': 345, 'JD Unadkat': 346, 
                                           'Mohammed Shami': 347, 'BMAJ Mendis': 348, 'R Rampaul': 349, 'CH Morris': 350, 
                                           'SV Samson': 351, 'SMSM Senanayake': 352, 'BJ Rohrer': 353, 'KL Rahul': 354, 
                                           'Q de Kock': 355, 'R Dhawan': 356, 'MG Johnson': 357, 'A Chandila': 358, 
                                           'LJ Wright': 359, 'CM Gautam': 360, 'X Thalaivan Sargunam': 361, 'DJG Sammy': 362, 
                                           'A Mukund': 363, 'KW Richardson': 364, 'UA Birla': 365, 'Parvez Rasool': 366, 
                                           'Sandeep Sharma': 367, 'S Kaul': 368, 'Sachin Baby': 369, 'PV Tambe': 370, 
                                           'NM Coulter-Nile': 371, 'MM Sharma': 372, 'CJ Anderson': 373, 'NJ Maddinson': 374, 
                                           'AR Patel': 375, 'JJ Bumrah': 376, 'JDS Neesham': 377, 'BW Hilfenhaus': 378, 
                                           'TG Southee': 379, 'MA Starc': 380, 'YS Chahal': 381, 'BR Dunk': 382, 'RR Rossouw': 383, 
                                           'R Shukla': 384, 'LMP Simmons': 385, 'VH Zol': 386, 'Imran Tahir': 387, 'BCJ Cutting': 388, 
                                           'BE Hendricks': 389, 'S Gopal': 390, 'M de Lange': 391, 'R Tewatia': 392, 'JO Holder': 393, 
                                           'Karanveer Singh': 394, 'DJ Muthuswami': 395, 'SS Iyer': 396, 'DJ Hooda': 397, 
                                           'Anureet Singh': 398, 'KS Williamson': 399, 'SA Abbott': 400, 'J Suchith': 401, 
                                           'RG More': 402, 'D Wiese': 403, 'HH Pandya': 404, 'SN Khan': 405,  'MJ McClenaghan': 406, 
                                           'SN Thakur': 407, 'JC Buttler': 408, 'CR Brathwaite': 409, 'MP Stoinis': 410,
                                           'Ishan Kishan': 411, 'C Munro': 412, 'P Sahu': 413, 'KH Pandya': 414, 'AD Nath': 415, 
                                           'KJ Abbott': 416, 'TM Head': 417, 'NS Naik': 418, 'RR Pant': 419, 'SW Billings': 420, 
                                           'KC Cariappa': 421, 'S Kaushik': 422, 'PSP Handscomb': 423, 'Swapnil Singh': 424, 
                                           'J Yadav': 425, 'UT Khawaja': 426, 'HM Amla': 427, 'F Behardien': 428, 'BB Sran': 429, 
                                           'N Rana': 430, 'MJ Guptill': 431, 'ER Dwivedi': 432, 'CJ Jordan': 433, 'TS Mills': 434, 
                                           'A Choudhary': 435, 'BA Stokes': 436, 'JJ Roy': 437, 'Vishnu Vinod': 438, 'PJ Cummins': 439, 
                                           'Basil Thampi': 440, 'CR Woakes': 441, 'A Zampa': 442, 'RA Tripathi': 443, 'DL Chahar': 444, 
                                           'Rashid Khan': 445, 'V Shankar': 446, 'RD Chahar': 447, 'LH Ferguson': 446, 
                                           'C de Grandhomme': 447, 'AF Milne': 448, 'S Badree': 449, 'Mohammad Nabi': 450, 
                                           'AJ Tye': 451, 'K Rabada': 452, 'Kuldeep Yadav': 453, 'DM Bravo': 454, 'AR Bawne': 455, 
                                           'SP Jackson': 456, 'Ankit Soni': 457, 'Washington Sundar': 458, 'TA Boult': 459, 
                                           'E Lewis': 460, 'MA Wood': 461, 'RK Singh': 462, 'DJM Short': 463, 'K Gowtham': 464, 
                                           'TK Curran': 465, 'M Markande': 466, 'B Stanlake': 467, 'Mujeeb Ur Rahman': 468, 
                                           'A Dananjaya': 469, 'Shubman Gill': 470, 'Shivam Mavi': 471, 'Mohammed Siraj': 472, 
                                           'H Klaasen': 473, 'JC Archer': 474, 'PP Shaw': 475, 'LE Plunkett': 476, 
                                           'Mustafizur Rahman': 477, 'AD Hales': 478, 'MK Lomror': 479, 'DR Shorey': 480, 'MM Ali': 481, 
                                           'M Prasidh Krishna': 482, 'P Chopra': 483, 'JPR Scantlebury-Searles': 484, 
                                           'Abhishek Sharma': 485, 'IS Sodhi': 486, 'SO Hetmyer': 487, 'S Dube': 488, 
                                           'NA Saini': 489, 'JM Bairstow': 490, 'Rasikh Salam': 491, 'KMA Paul': 492, 
                                           'N Pooran': 493, 'P Ray Barman': 494, 'Avesh Khan': 495, 'SM Curran': 496, 
                                           'GC Viljoen': 497, 'AS Joseph': 498, 'HF Gurney': 499, 'SD Lad': 500, 
                                           'R Parag': 501, 'LS Livingstone': 502, 'RK Bhui': 503, 'AJ Turner': 504, 
                                           'SE Rutherford': 505, 'Harpreet Brar': 504, 'Y Prithvi Raj': 505, 'M Ashwin': 506, 
                                           'MJ Santner': 507, 'P Simran Singh': 508, 'JL Pattinson': 509, 'T Banton': 510, 
                                           'D Padikkal': 511, 'YBK Jaiswal': 512, 'TU Deshpande': 513, 'A Nortje': 514, 
                                           'Abdul Samad': 515, 'PK Garg': 516, 'T Natarajan': 517, 'Ravi Bishnoi': 518, 
                                           'JR Philippe': 519, 'Kartik Tyagi': 520, 'I Udana': 521, 'KL Nagarkoti': 522, 
                                           'Shahbaz Ahmed': 523, 'RD Gaikwad': 524, 'AT Carey': 525, 'CV Varun': 526, 
                                           'N Jagadeesan': 527, 'P Dubey': 528, 'SS Cottrell': 529})
df.head()

In [ ]:
df['bowler'] = df['bowler'].map({'AA Noffke': 0, 'Z Khan': 1, 'JH Kallis': 2, 'SB Joshi': 3, 'CL White': 4, 'P Kumar': 5, 
                                 'AB Agarkar': 6, 'SC Ganguly': 7, 'LR Shukla': 8, 'I Sharma': 9, 'AB Dinda': 10, 'JDP Oram': 11, 
                                 'JR Hopes': 12, 'MS Gony': 13, 'K Goel': 14, 'PP Chawla': 15, 'WA Mota': 16, 'IK Pathan': 17, 
                                 'S Sreesanth': 18, 'B Lee': 19, 'M Muralitharan': 20, 'P Amarnath': 21, 'Joginder Sharma': 22, 
                                 'GD McGrath': 23, 'B Geeves': 24, 'SK Warne': 25, 'YK Pathan': 26, 'D Salunkhe': 27, 
                                 'SR Watson': 28, 'SK Trivedi': 29, 'DL Vettori': 30, 'MF Maharoof': 31, 'MM Patel': 32, 
                                 'R Bhatia': 33, 'B Akhil': 34, 'R Vinay Kumar': 35, 'A Nehra': 36, 'SM Pollock': 37, 
                                 'DS Kulkarni': 38, 'AM Nayar': 39, 'Harbhajan Singh': 40, 'ST Jayasuriya': 41, 'WPUJC Vaas': 42, 
                                 'RP Singh': 43, 'SB Styris': 44, 'SB Bangar': 45, 'DJ Hussey': 46, 'Mohammad Hafeez': 47, 
                                 'M Kartik': 48, 'PP Ojha': 49, 'A Symonds': 50, 'Pankaj Singh': 51, 'Mohammad Asif': 52, 
                                 'VY Mahesh': 53, 'Shahid Afridi': 54, 'MA Khote': 55, 'DJ Bravo': 56, 'VS Yeligati': 57, 
                                 'D Kalyankrishna': 58, 'VRV Singh': 59, 'A Kumble': 60, 'Sohail Tanvir': 61, 'DNT Zoysa': 62, 
                                 'SD Chitnis': 63, 'Shoaib Malik': 64, 'Yuvraj Singh': 65, 'DW Steyn': 66, 'JA Morkel': 67, 
                                 'CRD Fernando': 68, 'V Sehwag': 69, 'V Kohli': 70, 'Gagandeep Singh': 71, 'Y Venugopal Rao': 72, 
                                 'Umar Gul': 73, 'M Ntini': 74, 'PJ Sangwan': 75, 'DP Vijaykumar': 76, 'DB Ravi Teja': 77, 
                                 'LPC Silva': 78, 'DJ Thornely': 79, 'RR Raje': 80, 'SK Raina': 81, 'S Vidyut': 82, 'L Balaji': 83,
                                 'CK Kapugedera': 84, 'DR Smith': 85, 'AD Mascarenhas': 86, 'A Mishra': 87, 'Shoaib Akhtar': 88, 
                                 'Iqbal Abdulla': 89, 'RR Powar': 90, 'PM Sarvesh Kumar': 91, 'Abdur Razzak': 92, 'RA Jadeja': 93, 
                                 'TM Dilshan' 'RG Sharma': 94, 'A Nel': 95, 'BAW Mendis': 99, 'A Flintoff': 100, 'T Thushara': 101, 
                                 'SL Malinga': 102, 'T Henderson': 103, 'Kamran Khan': 104, 'JD Ryder': 105, 'DP Nannes': 106, 
                                 'AM Salvi': 107, 'YA Abdulla': 108, 'VS Malik': 109, 'Harmeet Singh': 110, 'FH Edwards': 111, 
                                 'MC Henriques': 112, 'CH Gayle': 113, 'KP Pietersen': 114, 'LRPL Taylor': 115, 'RS Bopara': 116, 
                                 'MK Tiwary': 117, 'Anureet Singh': 118, 'RR Bose': 119, 'KP Appanna': 120, 'Shoaib Ahmed': 121, 
                                 'BJ Hodge': 122, 'SR Tendulkar': 123, 'RE van der Merwe': 124, 'JP Duminy': 125, 'S Tyagi': 126, 
                                 'GR Napier': 127, 'SM Harwood': 128, 'AS Raut': 129, 'SB Jakati': 130, 'D du Preez': 131, 
                                 'TL Suman': 132, 'RJ Harris': 133, 'A Singh': 134, 'M Morkel': 135, 'LA Carseldine': 136,
                                 'A Mithun': 137, 'C Nanda': 138, 'SS Sarkar': 139, 'AD Mathews': 140, 'AM Rahane': 141, 
                                 'J Botha': 142, 'Mashrafe Mortaza': 143, 'AB McDonald': 144, 'Y Nagar': 145, 'CK Langeveldt': 146,
                                 'R Ashwin': 147, 'RA Shaikh': 148, 'Jaskaran Singh': 149, 'AA Jhunjhunwala': 150, 'A Uniyal': 151, 
                                 'SW Tait': 152, 'AG Murtaza': 153, 'R Sathish': 154, 'R McLaren': 155, 'RS Gavaskar': 156, 
                                 'JM Kemp': 157, 'S Ladda': 158, 'Bipul Sharma': 159, 'SE Bond': 160, 'KA Pollard': 161, 
                                 'S Narwal': 162, 'UT Yadav': 163, 'SJ Srivastava': 164, 'AC Voges': 165, 'R Sharma': 166, 
                                 'J Theron': 167, 'NLTC Perera': 168, 'KAJ Roach': 169, 'PD Collingwood': 170, 'C Ganapathy': 171, 
                                 'MB Parmar': 172, 'DE Bollinger': 173, 'SB Wagh': 174, 'JD Unadkat': 175, 'AP Dole': 176, 
                                 'AN Ahmed' : 177, 'FY Fazal': 178, 'MR Marsh': 179, 'L Ablish': 180, 'S Sriram': 181, 
                                 'AJ Finch': 182, 'STR Binny': 183, 'ND Doshi': 184, 'S Randiv': 185, 'TG Southee': 186, 
                                 'AL Menaria': 187, 'DT Christian': 188, 'AUK Pathan': 189, 'RV Gomez': 190, 'JEC Franklin': 191, 
                                 'WD Parnell': 192, 'M Manhas': 193, 'AC Thomas': 194, 'NJ Rimmington': 195, 'RN ten Doeschate': 196, 
                                 'S Nadeem': 197, 'BA Bhatt': 198, 'R Ninan': 199, 'JJ van der Wath': 200, 'S Aravind': 201, 
                                 'Shakib Al Hasan': 202, 'PC Valthaty': 203, 'S Dhawan': 204, 'J Syed Mohammad': 205, 
                                 'VR Aaron': 206, 'JE Taylor': 207, 'NL McCullum': 208, 'KMDN Kulasekara': 209, 
                                 'P Parameswaran': 210, 'I Malhotra': 211, 'B Chipli': 212, 'B Kumar': 213, 'AA Kazi': 214, 
                                 'Anand Rajan': 215, 'AA Chavan': 216, 'P Prasanth': 217, 'Y Gnaneswara Rao': 218, 
                                 'S Rana': 219, 'JP Faulkner': 220, 'SS Mundhe': 221, 'RW Price': 222, 'GJ Maxwell': 223, 
                                 'M de Lange': 224, 'MN Samuels': 225, 'KK Cooper': 226, 'DAJ Bracewell': 227, 'HV Patel': 228,
                                 'Ankit Sharma': 229, 'DJ Harris': 230, 'TP Sudhindra': 231, 'F du Plessis': 232, 'SP Narine': 233, 
                                 'GB Hogg': 234, 'RR Bhatkal': 235, 'P Awana': 236, 'V Pratap Singh': 237, 'CJ McKay': 238, 
                                 'A Ashish Reddy': 239, 'AD Russell': 240, 'Azhar Mahmood': 241, 'A Chandila': 242, 'P Negi': 243,
                                 'LJ Wright': 244, 'RJ Peterson': 245, 'R Shukla': 246, 'MJ Clarke': 247, 'BW Hilfenhaus': 248, 
                                 'SPD Smith': 249, 'K Upadhyay': 250, 'Sunny Gupta': 251, 'MG Johnson' : 252, 'JJ Bumrah': 253, 
                                 'S Badree': 254, 'AS Rajpoot': 255, 'B Laughlin': 256, 'GH Vihari': 257, 'Mohammed Shami': 258, 
                                 'BMAJ Mendis': 259, 'CH Morris': 260, 'Harmeet Singh (2)': 261, 'S Kaul': 262, 'R Dhawan': 263, 
                                 'MM Sharma': 264, 'R Rampaul': 265, 'SMSM Senanayake': 266, 'KV Sharma': 267, 'JO Holder': 268, 
                                 'IC Pandey': 269, 'Sandeep Sharma': 270, 'YS Chahal': 271, 'DJG Sammy': 272, 
                                 'KW Richardson': 273, 'P Suyal': 274, 'PV Tambe': 275, 'BJ Rohrer': 276, 
                                 'Parvez Rasool': 277, 'MG Neser': 278, 'NM Coulter-Nile': 279, 'AC Gilchrist': 280, 
                                 'CJ Anderson': 281, 'MA Starc': 282, 'JDS Neesham': 283, 'AR Patel': 284, 'M Vijay': 285, 
                                 'R Tewatia': 286, 'SA Yadav': 287, 'Shivam Sharma': 288, 'Imran Tahir': 289, 'V Shankar': 290, 
                                 'LMP Simmons': 291, 'BCJ Cutting': 292, 'S Gopal': 293, 'K Santokie': 294, 'BE Hendricks': 295, 
                                 'PJ Cummins': 296, 'JW Hastings': 297, 'Karanveer Singh': 298, 'DJ Muthuswami': 299, 'TA Boult': 300, 
                                 'SA Abbott': 301, 'KC Cariappa': 302, 'DJ Hooda': 303, 'J Suchith': 304, 'RG More': 305, 
                                 'MJ McClenaghan': 306, 'D Wiese': 307, 'HH Pandya': 308, 'SN Thakur': 309, 'GS Sandhu': 310, 
                                 'Gurkeerat Singh': 311, 'J Yadav': 312, 'BB Sran': 313, 'M Ashwin': 314, 'CR Brathwaite': 315, 
                                 'C Munro': 316, 'MP Stoinis': 317, 'P Sahu': 318, 'Mustafizur Rahman': 319, 'AF Milne': 320, 
                                 'Kuldeep Yadav': 321, 'KH Pandya': 322, 'KJ Abbott': 323, 'T Shamsi': 324, 'S Kaushik': 325, 
                                 'SM Boland': 326, 'Swapnil Singh': 327, 'CJ Jordan': 328, 'A Zampa': 329, 'Sachin Baby': 330, 
                                 'N Rana': 331, 'DL Chahar': 332, 'KS Williamson': 333, 'TM Head': 334, 'A Choudhary': 335, 
                                 'TS Mills': 336, 'Rashid Khan': 337, 'BA Stokes': 338, 'CR Woakes': 339, 'T Natarajan': 340, 
                                 'RD Chahar': 341, 'B Stanlake': 342, 'Basil Thampi': 343, 'Tejas Baroka': 344, 'C de Grandhomme': 345, 
                                 'AJ Tye': 346, 'LH Ferguson': 347, 'Mohammad Nabi': 348,'Mohammed Siraj': 349, 'K Rabada': 350, 
                                 'Washington Sundar': 351, 'SS Agarwal': 352, 'NB Singh': 353, 'RA Tripathi': 354, 'Ankit Soni': 355, 
                                 'MJ Henry': 356, 'Avesh Khan': 357, 'MA Wood': 358, 'M Markande': 359, 'Mujeeb Ur Rahman': 360, 
                                 'K Khejroliya': 361, 'K Gowtham': 362, 'DJM Short': 363, 'TK Curran': 364, 'A Dananjaya': 365, 
                                 'Shivam Mavi': 366, 'JC Archer': 367, 'LE Plunkett': 368, 'IS Sodhi': 369, 'MK Lomror': 370, 
                                 'KM Asif': 371, 'L Ngidi': 372, 'DJ Willey': 373, 'M Prasidh Krishna': 374, 'MM Ali': 375, 
                                 'JPR Scantlebury-Searles': 376, 'S Lamichhane': 377, 'CJ Dala': 378, 'KK Ahmed': 379, 'NA Saini': 380, 
                                 'S Dube': 381, 'Rasikh Salam': 382, 'KMA Paul': 383, 'SM Curran': 384, 'CV Varun': 385, 
                                 'GC Viljoen': 386, 'Mandeep Singh': 387, 'P Ray Barman': 388, 'MJ Santner': 389, 'JP Behrendorff': 390, 
                                 'SC Kuggeleijn': 391, 'AS Joseph': 392, 'HF Gurney': 393, 'S Midhun': 394, 'R Parag': 395, 
                                 'LS Livingstone': 396, 'SN Khan': 397, 'Abhishek Sharma': 398, 'Arshdeep Singh': 399, 
                                 'Harpreet Brar': 400, 'SE Rutherford': 401, 'Y Prithvi Raj': 402, 'O Thomas': 403, 'AS Roy': 404, 
                                 'S Sandeep Warrier': 405, 'JL Pattinson': 406, 'SS Cottrell': 407, 'A Nortje': 408, 
                                 'Ravi Bishnoi': 409, 'I Udana': 410, 'TU Deshpande': 411, 'KL Nagarkoti': 412, 
                                 'Kartik Tyagi': 413, 'DR Sams': 414, 'Shahbaz Ahmed': 415, 'Abdul Samad': 416, 
                                 'JR Hazlewood': 417, 'DA Warner': 418, 'CJ Green': 419, 'P Dubey': 420, 'Monu Kumar': 421})
df.head()

In [ ]:
df['dismissal_kind'] = df['dismissal_kind'].map({'caught': 0, 'bowled': 1, 'run out': 2, 'lbw': 3, 'stumped': 4, 'caught and bowled': 5, 
                                                'hit wicket': 6, 'retired hurt': 7, 'obstructing the field': 8})
df.head()

In [ ]:
df['player_dismissed'] = df['player_dismissed'].map({'RT Ponting': 0, 'DJ Hussey': 1, 'SC Ganguly': 2, 'MV Boucher': 3, 
                                                     'B Akhil': 4, 'CL White': 5, 'AA Noffke': 6, 'Z Khan': 7, 'SB Joshi': 8, 
                                                     'JH Kallis': 9, 'W Jaffer': 10, 'R Dravid': 11, 'V Kohli': 12, 
                                                     'SK Raina': 13, 'JDP Oram': 14, 'PA Patel': 15, 'ML Hayden': 16, 
                                                     'MS Dhoni': 17, 'JR Hopes': 18, 'Yuvraj Singh': 19, 'KC Sangakkara': 20, 
                                                     'K Goel': 21, 'SK Warne': 22, 'V Sehwag': 23, 'T Kohli': 24, 
                                                     'YK Pathan': 25, 'SR Watson': 26, 'DS Lehmann': 27, 'M Kaif': 28, 
                                                     'M Rawat': 29, 'RA Jadeja': 30, 'DJ Thornely': 31, 'L Ronchi': 32, 
                                                     'ST Jayasuriya': 33, 'PR Shah': 34, 'RV Uthappa': 35, 'AM Nayar': 36, 
                                                     'SM Pollock': 37, 'S Chanderpaul': 38, 'LRPL Taylor': 39, 
                                                     'BB McCullum': 40, 'WP Saha': 41, 'AS Yadav': 42, 'A Symonds': 43, 
                                                     'WPUJC Vaas': 44, 'SB Bangar': 45, 'PP Ojha': 46, 'Y Venugopal Rao': 47, 
                                                     'VVS Laxman': 48, 'AC Gilchrist': 49, 'RG Sharma': 50, 'SB Styris': 51, 
                                                     'Mohammad Hafeez': 52, 'Kamran Akmal': 53, 'DPMD Jayawardene': 54, 
                                                     'IK Pathan': 55, 'B Lee': 56, 'S Sohal': 57, 'G Gambhir': 58, 
                                                     'Shahid Afridi': 59, 'MEK Hussey': 60, 'Harbhajan Singh': 61, 
                                                     'DJ Bravo': 62, 'MA Khote': 63, 'GC Smith': 64, 'D Salunkhe': 65, 
                                                     'PP Chawla': 66, 'S Sreesanth': 67, 'VRV Singh': 68, 'RR Sarwan': 69, 
                                                     'SS Tiwary': 70, 'P Kumar': 71, 'M Kartik': 72, 'LR Shukla': 73, 
                                                     'AB Agarkar': 74, 'AM Rahane': 75, 'SM Katich': 76, 'TM Srivastava': 77, 
                                                     'S Dhawan': 78, 'Shoaib Malik': 79, 'KD Karthik': 80, 'R Bhatia': 81, 
                                                     'MK Tiwary': 82, 'MF Maharoof': 83, 'B Chipli': 84, 'DW Steyn': 85, 
                                                     'DB Das': 86, 'MK Pandey': 87, 'HH Gibbs': 88, 'DNT Zoysa': 89, 
                                                     'D Kalyankrishna': 90, 'SA Asnodkar': 91, 'BJ Hodge': 92, 
                                                     'Salman Butt': 93, 'Umar Gul': 94, 'AB Dinda': 95, 'SP Fleming': 96, 
                                                     'S Vidyut': 97, 'JA Morkel': 98, 'LPC Silva': 99, 'Misbah-ul-Haq': 100, 
                                                     'DB Ravi Teja': 101, 'RP Singh': 102, 'SE Marsh': 103, 
                                                     'AB de Villiers': 104, 'YV Takawale': 105, 'Mohammad Asif': 106, 
                                                     'GD McGrath': 107, 'PJ Sangwan': 108, 'S Badrinath': 109, 
                                                     'Joginder Sharma': 110, 'M Muralitharan': 111, 'M Ntini': 112, 
                                                     'DT Patil': 113, 'A Kumble': 114, 'S Anirudha': 115, 
                                                     'Sohail Tanvir': 116, 'SK Trivedi': 117, 'CK Kapugedera': 118, 
                                                     'A Chopra': 119, 'J Arunkumar': 120, 'T Taibu': 121, 
                                                     'R Vinay Kumar': 122, 'SP Goswami': 123, 'AD Mascarenhas': 124, 
                                                     'A Mishra': 125, 'VY Mahesh': 126, 'TM Dilshan': 127, 
                                                     'SR Tendulkar': 128, 'Younis Khan': 129, 'Niraj Patel': 130, 
                                                     'Shoaib Akhtar': 131, 'I Sharma': 132, 'H Das': 133, 'DR Smith': 134, 
                                                     'SD Chitnis': 135, 'A Nehra': 136, 'VS Yeligati': 137, 'MS Gony': 138, 
                                                     'L Balaji': 139, 'LA Pomersbach': 140, 'A Nel': 141, 'A Mukund': 142, 
                                                     'WA Mota': 143, 'JP Duminy': 144, 'A Flintoff': 145, 
                                                     'KP Pietersen': 146, 'JD Ryder': 147, 'T Henderson': 148, 
                                                     'MM Patel': 150, 'Kamran Khan': 151, 'RS Bopara': 152, 
                                                     'MC Henriques': 153, 'CH Gayle': 154, 'R Bishnoi': 155, 
                                                     'KV Sharma': 156, 'PC Valthaty': 157, 'RJ Quiney': 158, 
                                                     'Yashpal Singh': 159, 'Pankaj Singh': 160, 'FH Edwards': 161, 
                                                     'AS Raut': 162, 'AN Ghosh': 163, 'BAW Mendis': 164, 'DL Vettori': 165, 
                                                     'RE van der Merwe': 166, 'RR Powar': 167, 'AA Bilakhia': 168, 
                                                     'TL Suman': 169, 'Shoaib Ahmed': 170, 'GR Napier': 171, 
                                                     'MN van Wyk': 172, 'LA Carseldine': 173, 'NV Ojha': 174, 
                                                     'M Vijay': 175, 'DA Warner': 176, 'SB Jakati': 177, 'M Manhas': 178, 
                                                     'RJ Harris': 179, 'D du Preez': 180, 'RR Raje': 181,'DS Kulkarni': 182, 
                                                     'M Morkel': 183, 'AD Mathews': 184, 'S Tyagi': 185, 'SL Malinga': 186, 
                                                     'C Nanda': 187, 'J Botha': 188, 'A Singh': 189, 'GJ Bailey': 190, 
                                                     'AB McDonald': 191, 'Y Nagar': 192, 'SS Shaikh': 193, 'R Ashwin': 194, 
                                                     'T Thushara': 195, 'Mohammad Ashraful': 196, 'Anirudh Singh': 197, 
                                                     'CA Pujara': 198, 'AP Tare': 199, 'AT Rayudu': 200, 'R Sathish': 201, 
                                                     'AA Jhunjhunwala': 202, 'P Dogra': 203, 'A Uniyal': 204, 
                                                     'MS Bisla': 205, 'EJG Morgan': 206, 'JM Kemp': 207, 'OA Shah': 208, 
                                                     'RS Gavaskar': 209, 'SE Bond': 210, 'KA Pollard': 211, 
                                                     'DP Nannes': 212, 'DR Martyn': 213, 'S Narwal': 214, 'MJ Lumb': 215, 
                                                     'AB Barath': 216, 'FY Fazal': 217, 'MD Mishra': 218, 'S Ladda': 219, 
                                                     'J Theron': 220, 'SJ Srivastava': 221, 'R Sharma': 222, 
                                                     'Mandeep Singh': 223, 'AC Voges': 224, 'Jaskaran Singh': 226, 
                                                     'KAJ Roach': 227, 'PD Collingwood': 228, 'CK Langeveldt': 229, 
                                                     'VS Malik': 230, 'KM Jadhav': 231, 'SW Tait': 232, 'A Mithun': 233, 
                                                     'AP Dole': 234, 'Harmeet Singh': 235, 'R McLaren': 236, 
                                                     'S Sriram': 237, 'KP Appanna': 238, 'C Madan': 239, 'AG Paunikar': 240, 
                                                     'MR Marsh': 241, 'AJ Finch': 242, 'STR Binny': 243, 'B Sumanth': 244, 
                                                     'IR Jaggi': 245, 'DT Christian': 246, 
                                                     'MA Agarwal': 247, 'RV Gomez': 248, 'UT Yadav': 250, 'UBT Chand': 251, 
                                                     'DJ Jacobs': 252, 'Sunny Singh': 253, 'AL Menaria': 254, 
                                                     'AUK Pathan': 255, 'WD Parnell': 256, 'JJ van der Wath': 257, 
                                                     'R Ninan': 258, 'MS Wade': 259, 'JEC Franklin': 260, 'TD Paine': 261, 
                                                     'SB Wagh': 262, 'Shakib Al Hasan': 263, 'S Randiv': 264, 
                                                     'BJ Haddin': 265, 'NLTC Perera': 266, 'JE Taylor': 267, 
                                                     'NL McCullum': 268, 'RN ten Doeschate': 269, 'TR Birt': 270, 
                                                     'M Klinger': 271, 'Harpreet Singh': 272, 'Bipul Sharma': 273, 
                                                     'AC Blizzard': 274, 'CA Ingram': 275, 'S Nadeem': 276, 
                                                     'DH Yagnik': 277, 'KB Arun Karthik': 278, 'AA Chavan': 279, 
                                                     'ND Doshi': 280, 'CJ Ferguson': 281, 'Y Gnaneswara Rao': 282, 
                                                     'S Rana': 283, 'BA Bhatt': 284, 'F du Plessis': 285, 'RE Levi': 286, 
                                                     'DE Bollinger': 287, 'SA Yadav': 288, 'MN Samuels': 289, 
                                                     'B Kumar': 290, 'SPD Smith': 291, 'GJ Maxwell': 292, 'HV Patel': 293, 
                                                     'DJ Harris': 294, 'Ankit Sharma': 295, 'SP Narine': 296, 
                                                     'JP Faulkner': 297, 'KK Cooper': 298, 'GB Hogg': 299, 
                                                     'RR Bhatkal': 300, 'CJ McKay': 301, 'N Saini': 302, 'DA Miller': 303, 
                                                     'P Negi': 304, 'Azhar Mahmood': 305, 'AC Thomas': 306, 
                                                     'RJ Peterson': 307, 'A Ashish Reddy': 308, 'V Pratap Singh': 309, 
                                                     'MJ Clarke': 310, 'Gurkeerat Singh': 311, 'VR Aaron': 312, 
                                                     'AP Majumdar': 313, 'PA Reddy': 314, 'AD Russell': 315, 
                                                     'CA Lynn': 316, 'Sunny Gupta': 317, 'MC Juneja': 318, 'KK Nair': 319, 
                                                     'GH Vihari': 320, 'MDKJ Perera': 321, 'B Laughlin': 322, 
                                                     'Mohammed Shami': 323, 'BMAJ Mendis': 324, 'M Vohra': 325, 
                                                     'R Rampaul': 326, 'P Awana': 327, 'SMSM Senanayake': 328, 
                                                     'BJ Rohrer': 329, 'KL Rahul': 330, 'J Syed Mohammad': 331, 
                                                     'BB Samantray': 332, 'Q de Kock': 333, 'R Dhawan': 334, 
                                                     'MG Johnson': 335, 'LJ Wright': 336, 'AG Murtaza': 337, 
                                                     'IC Pandey': 338, 'X Thalaivan Sargunam': 339, 'DJG Sammy': 340, 
                                                     'SV Samson': 341, 'KW Richardson': 342, 'CH Morris': 343, 
                                                     'MM Sharma': 344, 'CM Gautam': 345, 'UA Birla': 346, 
                                                     'Parvez Rasool': 347, 'Sachin Baby': 348, 'NM Coulter-Nile': 349, 
                                                     'CJ Anderson': 350, 'NJ Maddinson': 351, 'AR Patel': 352, 
                                                     'JDS Neesham': 353, 'PV Tambe': 354, 'TG Southee': 355, 
                                                     'MA Starc': 356, 'BR Dunk': 357, 'RR Rossouw': 358, 
                                                     'Shivam Sharma': 359, 'LMP Simmons': 360, 'VH Zol': 361, 
                                                     'Imran Tahir': 362, 'R Shukla': 363, 'BCJ Cutting': 364, 
                                                     'BE Hendricks': 365, 'M de Lange': 366, 'S Gopal': 367, 
                                                     'R Tewatia': 368, 'JO Holder': 369, 'JD Unadkat': 370, 
                                                     'SS Iyer': 371, 'DJ Hooda': 372, 'SA Abbott': 373, 'AN Ahmed': 374, 
                                                     'KS Williamson': 375, 'Iqbal Abdulla': 376, 'SN Khan': 377, 
                                                     'D Wiese': 378, 'HH Pandya': 379, 'MJ McClenaghan': 380, 
                                                     'Anureet Singh': 381, 'DJ Muthuswami': 382, 'PJ Cummins': 383, 
                                                     'Karanveer Singh': 384, 'Sandeep Sharma': 385, 'J Suchith': 386, 
                                                     'JC Buttler': 387, 'CR Brathwaite': 388, 'MP Stoinis': 389, 
                                                     'Ishan Kishan': 390, 'C Munro': 391, 'AD Nath': 392, 'MJ Guptill': 393, 
                                                     'P Sahu': 394, 'TM Head': 395, 'M Ashwin': 396, 'KH Pandya': 397, 
                                                     'NS Naik': 398, 'RR Pant': 399, 'SW Billings': 400, 
                                                     'KC Cariappa': 401, 'PSP Handscomb': 402, 'Swapnil Singh': 403, 
                                                     'J Yadav': 404, 'UT Khawaja': 405, 'HM Amla': 406, 
                                                     'JJ Bumrah': 407, 'A Zampa': 408, 'N Rana': 409, 'S Kaushik': 410, 
                                                     'F Behardien': 411, 'KJ Abbott': 412, 'ER Dwivedi': 413, 
                                                     'CJ Jordan': 414, 'S Aravind': 415, 'YS Chahal': 416, 'TS Mills': 417, 
                                                     'BA Stokes': 418, 'JJ Roy': 419, 'Vishnu Vinod': 420, 'CR Woakes': 421, 
                                                     'RA Tripathi': 422, 'DL Chahar': 423, 'V Shankar': 424, 
                                                     'Rashid Khan': 425, 'SN Thakur': 426, 'C de Grandhomme': 427, 
                                                     'AF Milne': 428, 'S Badree': 429, 'Mohammad Nabi': 430, 'K Rabada': 431, 
                                                     'AJ Tye': 432, 'Kuldeep Yadav': 433, 'SP Jackson': 434, 
                                                     'Ankit Soni': 435, 'A Choudhary': 436, 'AS Rajpoot': 437, 
                                                     'Washington Sundar': 438, 'E Lewis': 439, 'MA Wood': 440, 
                                                     'RK Singh': 441, 'DJM Short': 442, 'K Gowtham': 443, 'S Kaul': 444, 
                                                     'Mujeeb Ur Rahman': 445, 'Shivam Mavi': 446, 'Shubman Gill': 447, 
                                                     'TK Curran': 448, 'TA Boult': 449, 'H Klaasen': 450, 'RK Bhui': 451, 
                                                     'JC Archer': 452, 'PP Shaw': 453, 'LE Plunkett': 454, 
                                                     'M Markande': 455, 'Mustafizur Rahman': 456, 'Basil Thampi': 457, 
                                                     'BB Sran': 458, 'AD Hales': 459, 'MK Lomror': 460, 
                                                     'Mohammed Siraj': 461, 'DR Shorey': 462, 'MM Ali': 463, 
                                                     'P Chopra': 464, 'IS Sodhi': 465, 'Abhishek Sharma': 466, 
                                                     'JPR Scantlebury-Searles': 467, 'SO Hetmyer': 468, 'S Dube': 469, 
                                                     'NA Saini': 470, 'JM Bairstow': 471, 'KMA Paul': 472, 'N Pooran': 473, 
                                                     'P Ray Barman': 474, 'S Lamichhane': 475, 'SM Curran': 476, 
                                                     'GC Viljoen': 477, 'RD Chahar': 478, 'M Prasidh Krishna': 479, 
                                                     'SD Lad': 480, 'R Parag': 481, 'JL Denly': 482, 'LS Livingstone': 483, 
                                                     'KK Ahmed': 484, 'AJ Turner': 485, 'SE Rutherford': 486, 
                                                     'MJ Santner': 487, 'P Simran Singh': 488, 'JL Pattinson': 489, 
                                                     'T Banton': 490, 'D Padikkal': 491, 'YBK Jaiswal': 492, 
                                                     'RD Gaikwad': 493, 'TU Deshpande': 494, 
                                                     'Abdul Samad': 495, 'PK Garg': 496, 'JR Philippe': 497, 
                                                     'CV Varun': 498, 'KL Nagarkoti': 499, 'I Udana': 500, 
                                                     'Ravi Bishnoi': 501, 'AT Carey': 502, 'N Jagadeesan': 503, 
                                                     'A Nortje': 504, 'Kartik Tyagi': 505, 'SS Cottrell': 506, 
                                                     'Arshdeep Singh': 507, 'DR Sams': 508})
df.head()

In [ ]:
df['fielder'] = df['fielder'].map({'P Kumar': 0, 'CL White': 1, 'JH Kallis': 2, 'M Kartik': 3, 'RT Ponting': 4, 'WP Saha': 5, 
                                   'AB Agarkar,WP Saha': 6, 'BB McCullum': 7, 'IK Pathan': 8, 'KC Sangakkara': 9, 
                                   'Joginder Sharma': 10, 'P Amarnath': 11, 'PA Patel': 12, 'M Manhas': 13, 'B Geeves': 14, 
                                   'V Kohli': 15, 'Z Khan': 16, 'MV Boucher': 17, 'AM Nayar': 18, 'L Ronchi': 19, 
                                   'RG Sharma': 20, 'SB Styris': 21, 'AB Dinda': 22, 'DJ Hussey': 23, 'SC Ganguly': 24, 
                                   'Mohammad Hafeez,WP Saha': 25, 'LR Shukla': 26, 'Mohammad Hafeez': 27, 'A Symonds': 28, 
                                   'PP Chawla': 29, 'Kamran Akmal': 30, 'RA Jadeja,Kamran Akmal': 31, 'MK Tiwary,KD Karthik': 32, 
                                   'GD McGrath': 33, 'G Gambhir': 34, 'AC Gilchrist': 35, 'S Dhawan': 36, 'KD Karthik': 37, 
                                   'DJ Bravo': 38, 'S Badrinath': 39, 'Harbhajan Singh': 40, 'JDP Oram': 41, 'SK Raina': 42, 
                                   'RA Jadeja': 43, 'D Kalyankrishna': 44, 'VVS Laxman,RP Singh': 45, 'SB Bangar': 46, 
                                   'D Salunkhe': 47, 'DS Kulkarni': 48, 'DJ Bravo,L Ronchi': 49, 'A Nehra': 50, 
                                   'Harbhajan Singh,L Ronchi': 51, 'B Lee': 52, 'RR Sarwan': 53, 'AM Rahane (sub)': 54, 
                                   'Yuvraj Singh': 55, 'M Rawat,SR Watson': 56, 'MM Patel': 57, 'GC Smith': 58, 
                                   'M Rawat': 59, 'Sohail Tanvir,M Rawat': 60, 'A Kumble': 61, 'MS Dhoni,Joginder Sharma': 62, 
                                   'I Sharma': 63, 'M Muralitharan': 64, 'PP Ojha': 65, 'VVS Laxman': 66, 'V Sehwag,R Bhatia': 67, 
                                   'DPMD Jayawardene,KC Sangakkara': 68, 'W Jaffer,MV Boucher': 69, 'MS Dhoni': 70, 
                                   'MEK Hussey': 71, 'R Vinay Kumar (sub)': 72, 'R Dravid': 73, 'SK Raina,MS Dhoni': 74, 
                                   'Joginder Sharma,JDP Oram,MS Dhoni': 75, 'RV Uthappa': 76, 'AM Rahane,ST Jayasuriya': 77, 
                                   'PR Shah': 78, 'ST Jayasuriya': 79, 'V Sehwag': 80, 'LRPL Taylor': 81, 'VY Mahesh': 82, 
                                   'K Goel': 83, 'S Sreesanth': 84, 'Yuvraj Singh,KC Sangakkara': 85, 'SE Marsh': 86, 
                                   'SK Trivedi': 87, 'M Kaif': 88, 'RA Jadeja,M Rawat': 89, 'Umar Gul,WP Saha': 90, 
                                   'SR Watson': 91, 'T Kohli (sub)': 92, 'Shoaib Malik': 93, 'PJ Sangwan': 94, 'SP Goswami': 95, 
                                   'DW Steyn': 96, 'B Akhil': 97, 'LPC Silva': 98, 'W Jaffer': 99, 'R Vinay Kumar': 100, 
                                   'B Chipli (sub)': 101, 'AS Yadav': 102, 'DB Ravi Teja': 103, 'DB Das,DJ Hussey': 104, 
                                   'BJ Hodge': 105, 'JR Hopes': 106, 'SM Pollock': 107, 'AB de Villiers': 108, 
                                   'DJ Thornely': 109, 'SS Tiwary': 110, 'Sohail Tanvir': 111, 'SK Warne': 112, 
                                   'M Ntini': 113, 'U Kaul': 114, 'DPMD Jayawardene': 115, 'DT Patil': 116, 
                                   'RP Singh': 117, 'Shahid Afridi': 118, 'HH Gibbs': 119, 'YV Takawale': 120, 
                                   'YK Pathan': 121, 'SA Asnodkar': 122, 'MK Tiwary': 123, 'CK Kapugedera': 124, 
                                   'DW Steyn,MV Boucher': 125, 'M Rawat,Sohail Tanvir': 126, 'YV Takawale,CRD Fernando': 127, 
                                   'SR Tendulkar': 128, 'CRD Fernando': 129, 'JA Morkel': 130, 'SP Fleming': 131, 
                                   'S Vidyut': 132,'Salman Butt': 133, 'A Chopra': 134, 'Niraj Patel': 135, 
                                   'V Sehwag,KD Karthik': 136, 'PP Chawla,U Kaul': 137, 'T Taibu': 138, 
                                   'TM Dilshan,AB de Villiers': 139, 'MF Maharoof': 140, 'LR Shukla,WP Saha': 141, 
                                   'RR Raje': 142, 'Younis Khan': 143, 'YK Pathan,SR Watson': 144, 'WA Mota': 145, 
                                   'TM Dilshan': 146, 'AB de Villiers (sub)': 147, 'Shoaib Malik (sub)': 148, 
                                   'RV Uthappa,SM Pollock': 149, 'LA Pomersbach': 150, 'RR Powar': 151, 'Misbah-ul-Haq': 152, 
                                   'MS Gony': 153, 'R Bhatia': 154, 'B Chipli': 155, 'CL White,SP Goswami': 156, 
                                   'MF Maharoof,Shoaib Malik (sub),KD Karthik': 157, 'V Sehwag,MF Maharoof': 158, 
                                   'DR Smith': 159, 'Yuvraj Singh,PP Chawla': 160, 'TM Srivastava': 161, 
                                   'TM Srivastava,DPMD Jayawardene,U Kaul': 162, 'VRV Singh': 163, 'R Dravid,SP Goswami': 164, 
                                   'SD Chitnis': 165, 'MS Dhoni,PA Patel': 166, 'A Mukund': 167, 'S Badrinath,PA Patel': 168, 
                                   'Sohail Tanvir,Kamran Akmal': 169, 'T Kohli': 170, 'RA Jadeja (sub)': 171, 
                                   'SB Bangar,SB Styris': 172, 'Pankaj Singh': 173, 'MK Pandey': 174, 
                                   'RR Raje,ST Jayasuriya': 175, 'L Balaji': 176, 'MS Gony,PA Patel': 177, 
                                   'TM Dilshan,KD Karthik': 178, 'YK Pathan,M Rawat': 179, 'M Ntini,MS Gony': 180, 
                                   'ML Hayden': 181, 'T Thushara': 182, 'R Ashwin': 183, 'V Kohli,RV Uthappa': 184, 
                                   'JD Ryder': 185, 'KD Karthik,DL Vettori': 186, 'DL Vettori': 187, 
                                   'DP Nannes': 188, 'MC Henriques': 189, 'LR Shukla,BB McCullum': 190, 
                                   'Harmeet Singh': 191, 'R Bishnoi': 192, 'A Flintoff': 193, 'A Chopra,BB McCullum': 194, 
                                   'Yashpal Singh': 195, 'CH Gayle': 196, 'Y Venugopal Rao': 197, 
                                   'AC Gilchrist,Harmeet Singh': 198, 'KP Pietersen': 199, 'KV Sharma,RV Uthappa': 200, 
                                   'DA Warner (sub)': 201, 'V Sehwag,DL Vettori': 202, 'PJ Sangwan,DP Nannes': 203, 'AS Raut': 204, 
                                   'Kamran Khan,M Rawat': 205, 'Kamran Khan': 206, 'RS Bopara': 207, 'R Vinay Kumar,RV Uthappa': 208, 
                                   'DJ Bravo,Z Khan': 209, 'A Mishra': 210, 'RA Jadeja,SA Asnodkar': 211, 'Shoaib Ahmed': 212, 
                                   'BAW Mendis': 213, 'AN Ghosh': 214, 'JP Duminy': 215, 'MN van Wyk': 216, 
                                   'DL Vettori,KD Karthik': 217, 'AD Mascarenhas': 218, 'JP Duminy,PR Shah': 219, 'SL Malinga': 220, 
                                   'RE van der Merwe,MV Boucher': 221, 'SM Katich': 222, 'HH Gibbs,Y Venugopal Rao': 223, 
                                   'AB de Villiers,KD Karthik': 224, 'VS Malik': 225, 'M Vijay': 226, 'SB Jakati': 227, 
                                   'SM Harwood': 228, 'NV Ojha': 229, 'DJ Bravo,PR Shah': 230, 'RJ Harris': 231, 
                                   'V Kohli,MV Boucher': 232, 'M Morkel': 233, 'LA Carseldine': 234, 'DA Warner,KD Karthik': 235, 
                                   'DA Warner': 236, 'HH Gibbs,AC Gilchrist': 237, 'DB Ravi Teja,AC Gilchrist': 238, 
                                   'SM Katich,KC Sangakkara': 239, 'SL Malinga,Harbhajan Singh': 240, 'TL Suman,AC Gilchrist': 241, 
                                   'AC Gilchrist,Y Venugopal Rao': 242, 'AD Mathews': 243, 'B Akhil,P Kumar': 244, 
                                   'RE van der Merwe': 245, 'M Muralitharan,MS Dhoni': 246, 'S Tyagi': 247, 'SK Warne,MM Patel': 248, 
                                   'AM Rahane': 249, 'Harbhajan Singh,ST Jayasuriya': 250, 'KC Sangakkara,WA Mota': 251, 'S Sohal': 252, 
                                   'ML Hayden,MS Dhoni': 253, 'TL Suman': 254, 'AB Agarkar,BJ Hodge': 255, 'Mashrafe Mortaza': 256, 
                                   'LR Shukla (sub)': 257, 'KC Sangakkara,B Lee': 258, 'AM Salvi': 259, 'SS Shaikh': 260, 
                                   'J Botha,YK Pathan': 261, 'RJ Quiney': 262, 'Mohammad Ashraful': 263, 'TM Dilshan,PJ Sangwan': 264, 
                                   'MV Boucher,P Kumar': 265, 'Anirudh Singh': 266, 'P Dogra': 267, 'R Sathish': 268, 
                                   'DPMD Jayawardene,PP Chawla': 269, 'MS Bisla': 270, 'EJG Morgan': 271, 'FY Fazal (sub)': 272, 
                                   'G Gambhir,A Mishra': 273, 'A Mishra,KD Karthik': 274, 'Y Nagar (sub)': 275, 'AT Rayudu': 276, 
                                   'AP Tare': 277, 'R Dravid,MV Boucher': 278, 'JM Kemp': 279, 'Bipul Sharma': 280, 'CA Pujara': 281, 
                                   'AC Voges': 282, 'AS Raut (sub)': 283, 'AD Mathews,M Kartik': 284, 'KA Pollard': 285, 
                                   'WPUJC Vaas': 286, 'RP Singh,AC Gilchrist': 287, 'ML Hayden,JA Morkel': 288, 'GJ Bailey (sub)': 289, 
                                   'J Theron': 290, 'KP Appanna (sub)': 291, 'SJ Srivastava': 292, 'AA Jhunjhunwala': 293, 
                                   'SW Tait': 294, 'PP Chawla,IK Pathan': 295, 'MJ Lumb': 296, 'FY Fazal': 297, 
                                   'AA Jhunjhunwala,SK Warne': 298, 'A Mishra,DP Nannes': 299, 'UT Yadav,KD Karthik': 300, 
                                   'MS Dhoni,T Thushara': 301, 'SB Jakati,MS Dhoni': 302, 'AC Voges,NV Ojha': 303, 'R McLaren': 304, 
                                   'JP Duminy (sub)': 305, 'SL Malinga,AT Rayudu': 306, 'SK Raina,SB Jakati': 307, 
                                   'S Ladda,KD Karthik': 308, 'KAJ Roach': 309, 'MK Pandey,RV Uthappa': 310, 'S Narwal': 311, 
                                   'S Anirudha': 312, 'DE Bollinger': 313, 'KA Pollard,AT Rayudu': 314, 'M Kaif (sub)': 315, 
                                   'KP Appanna': 316, 'PD Collingwood': 317, 'AA Bilakhia': 318, 'AP Dole': 319, 
                                   'S Badrinath,MS Dhoni': 320, 'AA Jhunjhunwala,FY Fazal,NV Ojha': 321, 'R Bhatia,DL Vettori': 322, 
                                   'Iqbal Abdulla': 323, 'RP Singh,Harmeet Singh': 324, 'B Sumanth': 325, 'MD Mishra': 326, 
                                   'AB Agarkar': 327, 'RV Uthappa,JH Kallis': 328, 'RS Sodhi': 329, 'J Theron,KC Sangakkara': 330, 
                                   'K Goel,KC Sangakkara': 331, 'SS Tiwary,AT Rayudu': 332, 'S Dhawan (sub)': 333, 
                                   'ST Jayasuriya,Harbhajan Singh': 334, 'RG Sharma,AC Gilchrist': 335, 'DW Steyn,RV Uthappa': 336, 
                                   'KA Pollard,C Madan': 337, 'AG Murtaza': 338, 'JH Kallis,R Vinay Kumar': 339, 
                                   'AC Voges,Kamran Khan': 340, 'UT Yadav': 341, 'JD Unadkat,BB McCullum': 342, 'JD Unadkat': 343, 
                                   'MR Marsh,R Sharma': 344, 'AM Nayar,KA Pollard': 345, 'M Vijay,MS Dhoni,JA Morkel': 346, 
                                   'R Ashwin,MS Dhoni': 347, 'S Ladda': 348, 'AG Paunikar': 349, 'J Botha': 350, 'AL Menaria': 351, 
                                   'R Sathish,DJ Jacobs': 352, 'DJ Jacobs': 353, 'JEC Franklin': 354, 'R Sharma': 355, 
                                   'Yuvraj Singh,RV Uthappa': 356, 'RN ten Doeschate': 357, 'IR Jaggi': 358, 'PC Valthaty': 359, 
                                   'S Randiv': 360, 'S Randiv,S Anirudha': 361, 'WD Parnell,RV Uthappa': 362, 
                                   'RV Uthappa,NL McCullum (sub)': 363, 'WD Parnell': 364, 'RV Gomez,RA Jadeja': 365, 
                                   'DT Christian': 366, 'AG Paunikar,FY Fazal (sub)': 367, 'MA Agarwal': 368, 'R Ninan': 369, 
                                   'Sunny Singh': 370, 'AJ Finch': 371, 'SB Wagh,RV Uthappa': 372, 'SB Wagh': 373, 
                                   'Shakib Al Hasan,MS Bisla': 374, 'NLTC Perera': 375, 'AC Thomas': 376, 'KM Jadhav': 377, 
                                   'PA Patel,RA Jadeja': 378, 'DH Yagnik': 379, 'STR Binny,SR Watson': 380, 
                                   'AB de Villiers,S Aravind': 381, 'Y Nagar': 382, 'A Symonds,AN Ahmed': 383, 
                                   'KA Pollard,DJ Jacobs': 384, 'STR Binny': 385, 'J Botha,DH Yagnik': 386, 
                                   'KM Jadhav,PA Patel': 387, 'NL McCullum': 388, 'A Mukund (sub)': 389, 
                                   'A Mithun,AB de Villiers': 390, 'PA Patel,NLTC Perera': 391, 'RV Gomez': 392, 
                                   'SP Goswami,B Lee': 393, 'L Balaji,B Lee': 394, 'Y Nagar,NV Ojha': 395, 
                                   'Shakib Al Hasan (sub)': 396, 'Harpreet Singh': 397, 'J Syed Mohammad': 398, 
                                   'RA Jadeja,PA Patel': 399, 'SP Goswami,Iqbal Abdulla': 400, 'YK Pathan,SP Goswami': 401, 
                                   'EJG Morgan,SP Goswami': 402, 'DH Yagnik,SK Trivedi': 403, 'F du Plessis (sub)': 404, 
                                   'CL White,KC Sangakkara': 405, 'AC Blizzard': 406, 'PA Patel,R Vinay Kumar': 407, 
                                   'S Ladda,JH Kallis': 408, 'AB Agarkar,NV Ojha': 409, 'A Mithun': 410, 'AUK Pathan': 411, 
                                   'R Bhatia,MV Boucher': 412, 'TL Suman,AT Rayudu': 413, 'KB Arun Karthik (sub)': 414, 
                                   'M Kaif,AB de Villiers': 415, 'B Kumar': 416, 'WP Saha,R Ashwin': 417, 'CJ Ferguson': 418, 
                                   'Mandeep Singh,AC Gilchrist': 419, 'AB de Villiers,KB Arun Karthik': 420, 'KB Arun Karthik': 421, 
                                   'Y Nagar,IK Pathan': 422, 'DJ Hussey,AC Gilchrist': 423, 'DT Christian,Anand Rajan': 424, 
                                   'Mandeep Singh': 425, 'OA Shah': 426, 'RA Jadeja,BJ Hodge': 427, 'AA Chavan': 428, 
                                   'Y Gnaneswara Rao': 429, 'DT Christian,PP Ojha': 430, 'MS Dhoni,DE Bollinger': 431, 
                                   'Anand Rajan': 432, 'KH Devdhar': 433, 'AN Ahmed': 434, 'S Aravind': 435, 
                                   'RG Sharma,DS Kulkarni': 436, 'AN Ahmed,KD Karthik': 437, 'Harbhajan Singh,KD Karthik': 438, 
                                   'AJ Finch,NV Ojha': 439, 'KD Karthik,KA Pollard': 440, 'A Singh': 441, 'KK Cooper': 442, 
                                   'JP Faulkner': 443, 'KK Cooper,SP Goswami': 444, 'DAJ Bracewell': 445, 
                                   'DAJ Bracewell,GJ Maxwell': 446, 'HV Patel': 447, 'HV Patel,AB de Villiers': 448, 
                                   'DJ Harris': 449, 'Iqbal Abdulla,R Bhatia': 450, 'SPD Smith,M Kartik': 451, 
                                   'SPD Smith,RV Uthappa': 452, 'IK Pathan,DPMD Jayawardene': 453, 'KP Pietersen,NV Ojha': 454, 
                                   'V Sehwag,NV Ojha': 455, 'F du Plessis,R Ashwin': 456, 'RE Levi': 457, 
                                   'RR Bhatkal': 458, 'F du Plessis': 459, 'TM Dilshan (sub)': 460, 'SPD Smith': 461, 
                                   'P Awana': 462, 'S Nadeem': 463, 'RA Jadeja,R Ashwin': 464, 'DJ Bravo,KMDN Kulasekara': 465, 
                                   'G Gambhir,R Bhatia': 466, 'DB Das': 467, 'T Mishra': 468, 'AB de Villiers,R Vinay Kumar': 469, 
                                   'BB McCullum,MS Bisla': 470, 'KMDN Kulasekara': 471, 'N Saini': 472, 'AD Mathews,RV Uthappa': 473, 
                                   'SE Marsh,PP Chawla': 474, 'DA Miller': 475, 'DA Miller,N Saini': 476, 'JH Kallis,R Bhatia': 477, 
                                   'LA Pomersbach (sub)': 478, 'LJ Wright,RV Uthappa': 479, 'Azhar Mahmood': 480, 
                                   'DJ Harris (sub)': 481, 'NLTC Perera (sub)': 482, 'DPMD Jayawardene,NV Ojha': 483, 
                                   'P Dogra (sub)': 484, 'GB Hogg': 485, 'B Lee,BB McCullum': 486, 'A Mishra,PA Patel': 487, 
                                   'KP Appanna,AB de Villiers': 488, 'R Vinay Kumar,AB de Villiers': 489, 'DA Miller (sub)': 490, 
                                   'RJ Peterson': 491, 'RA Jadeja,MS Dhoni': 492, 'Gurkeerat Singh': 493, 'WP Saha (sub)': 494, 
                                   'V Kohli,J Syed Mohammad': 495, 'BB McCullum,SP Narine': 496, 'MJ Clarke': 497, 
                                   'A Ashish Reddy,J Theron': 498, 'KP Appanna,SS Tiwary': 499, 'DR Smith,KD Karthik': 500, 
                                   'S Anirudha,MS Dhoni': 501, 'BB McCullum,R Bhatia': 502, 'VR Aaron': 503, 
                                   'P Negi': 504, 'Harbhajan Singh,DR Smith': 505, 'A Chandila': 506, 'CA Lynn': 507, 
                                   'UBT Chand': 508, 'PJ Sangwan (sub)': 509, 'RA Jadeja,SB Jakati': 510, 'DA Warner,NV Ojha': 511, 
                                   'DJ Bravo,R Ashwin': 512, 'Shakib Al Hasan': 513, 'MS Bisla,SP Narine': 514, 'SP Narine': 515, 
                                   'MG Johnson': 516, 'JD Unadkat,KB Arun Karthik': 517, 'MR Marsh': 518, 'MN Samuels': 519, 
                                   'R Vinay Kumar,KB Arun Karthik': 520, 'Ankit Sharma': 521, 'A Ashish Reddy': 522, 
                                   'R Shukla': 523, 'Mohammed Shami,MS Bisla': 524, 'BMAJ Mendis': 525, 'R Dhawan': 526, 
                                   'CH Morris':  527, 'DT Christian (sub)': 528, 'KL Rahul': 529, 'PA Reddy': 530, 'GH Vihari': 531, 
                                   'AT Rayudu,PP Ojha': 532, 'SV Samson': 533, 'AC Gilchrist,P Kumar': 534, 'IK Pathan,M Morkel': 535, 
                                   'BB Samantray': 536, 'Q de Kock': 537, 'R Dravid,DH Yagnik': 538, 'MG Johnson,PP Ojha': 539, 
                                   'SK Raina,RA Jadeja': 540, 'BMAJ Mendis,KM Jadhav': 541, 'PA Reddy,NLTC Perera': 542, 
                                   'I Sharma,Q de Kock': 543, 'DJ Bravo,MS Dhoni': 544, 'MM Sharma': 545, 'SMSM Senanayake': 546, 
                                   'R Rampaul': 547, 'AD Russell': 548, 'M Vohra': 549, 'S Kaul': 550, 'BJ Rohrer': 551, 
                                   'BB McCullum (sub)': 552, 'DJG Sammy': 553, 'JH Kallis,MS Bisla': 554, 
                                   'AB de Villiers,J Syed Mohammad': 555, 'DB Das,L Balaji': 556, 'LJ Wright': 557, 
                                   'JP Faulkner,SV Samson': 558, 'V Kohli,AB de Villiers': 559, 'JA Morkel,R Ashwin': 560, 
                                   'KV Sharma': 561, 'R Sathish,Gurkeerat Singh': 562, 'M Muralitharan,KB Arun Karthik': 563, 
                                   'GH Vihari,DJG Sammy': 564, 'P Suyal': 565, 'UA Birla': 566, 'Sachin Baby': 567, 
                                   'RG Sharma,KD Karthik': 568, 'L Balaji,MS Bisla': 569, 'JO Holder': 570, 'K Upadhyay': 570, 
                                   'BJ Rohrer,KM Jadhav': 571, 'R Sathish,Harmeet Singh': 572, 'I Sharma,DJG Sammy': 573, 
                                   'NLTC Perera,PA Patel': 575, 'MS Bisla,JH Kallis': 576, 'AG Murtaza,RV Uthappa': 577, 
                                   'LJ Wright,RV Uthappa,AD Mathews': 578, 'SR Watson,JP Faulkner,DH Yagnik': 579, 'GJ Maxwell': 580, 
                                   'STR Binny,Sachin Baby,KK Cooper': 581, 'GJ Maxwell,KD Karthik' 'NM Coulter-Nile': 582, 
                                   'BB Samantray,PA Patel': 583, 'EJG Morgan,MS Bisla': 584, 'AM Rahane,JP Faulkner': 585, 
                                   'R Shukla (sub),DH Yagnik': 586, 'MM Sharma,CH Morris': 587, 'AR Patel (sub)': 588, 
                                   'CJ Anderson': 589, 'GJ Bailey': 590, 'KW Richardson': 591, 'YS Chahal': 592, 'S Rana': 593, 
                                   'NJ Maddinson': 594, 'WD Parnell (sub)': 595, 'Mohammed Shami': 596, 'AR Patel': 597, 
                                   'BW Hilfenhaus': 598, 'MA Starc': 599, 'CM Gautam': 600, 'AM Rahane,SV Samson': 601, 
                                   'YV Takawale,PA Patel': 602, 'TG Southee': 603, 'S Nadeem,Mohammed Shami': 604, 
                                   'AV Wankhade (sub)': 605, 'IC Pandey': 606, 'MK Pandey,MS Bisla': 607, 'SA Yadav': 608, 
                                   'SPD Smith,SV Samson': 609, 'KK Nair': 611, 'BR Dunk': 612, 'MS Dhoni,BW Hilfenhaus': 613, 
                                   'MC Henriques (sub)': 614, 'SK Raina,RA Jadeja,MM Sharma': 615, 'PA Patel,MA Starc': 616, 
                                   'LMP Simmons': 617, 'P Kumar,CM Gautam': 618, 'SA Yadav,RV Uthappa': 619, 'RR Rossouw (sub)': 620, 
                                   'RG Sharma,CM Gautam': 621, 'MS Dhoni,BB McCullum': 622, 'R Shukla,Q de Kock': 623, 
                                   'Shivam Sharma': 624, 'M Vohra,Sandeep Sharma': 625, 'BCJ Cutting': 626, 
                                   'NV Ojha,DJG Sammy': 627, 'K Santokie': 628, 'BE Hendricks': 629, 'Parvez Rasool': 630, 
                                   'Yuvraj Singh,PA Patel': 631, 'Sandeep Sharma': 632, 'V Kohli,YV Takawale': 633, 
                                   'SK Raina,P Negi': 634, 'KD Karthik,KM Jadhav': 635, 'Karanveer Singh': 636, 
                                   'RV Uthappa,R Vinay Kumar': 637, 'R Dhawan,GJ Bailey': 638, 'KK Nair,SV Samson': 639, 
                                   'PP Chawla,RV Uthappa': 640, 'BR Dunk (sub)': 641, 'GJ Maxwell,WP Saha': 642, 'SS Iyer': 643, 
                                   'MS Dhoni,A Nehra': 644, 'TA Boult': 645, 'M Morkel,Shakib Al Hasan': 646, 
                                   'KD Karthik,VR Aaron': 647, 'J Suchith': 648, 'KS Williamson': 649, 'RG More': 650, 
                                   'PJ Cummins': 651, 'Anureet Singh': 652, 'RN ten Doeschate,RV Uthappa': 653, 
                                   'RG Sharma,KA Pollard': 654, 'MS Bisla,AN Ahmed': 655, 'Imran Tahir': 656, 'R Tewatia': 657, 
                                   'RR Rossouw': 658, 'DR Smith,MS Dhoni': 659, 'J Yadav (sub)': 660, 'HV Patel,V Kohli': 661, 
                                   'D Wiese': 662, 'HH Pandya (sub)': 663, 'GH Vihari,NV Ojha': 664, 'JP Duminy,KM Jadhav': 665, 
                                   'DJ Hooda': 666, 'DJ Hooda,SV Samson': 667, 'TG Southee,SV Samson': 668, 
                                   'KA Pollard,R Vinay Kumar': 669, 'B Aparajith (sub)': 670, 'J Suchith,R Vinay Kumar': 671, 
                                   'P Negi,MS Dhoni': 672, 'G Gambhir,RN ten Doeschate': 673, 'Bipul Sharma,NV Ojha': 674, 
                                   'HH Pandya': 675, 'J Yadav': 676, 'V Kohli,KD Karthik': 677, 'Shakib Al Hasan,Azhar Mahmood': 678, 
                                   'HH Pandya,KA Pollard': 679, 'MJ McClenaghan,PA Patel': 680, 'UBT Chand (sub)': 681, 
                                   'STR Binny,JP Faulkner': 682, 'YS Chahal,KD Karthik': 683, 'SN Khan': 684, 'AF Milne': 685, 
                                   'GB Hogg,RV Uthappa': 686, 'C Munro': 687, 'TG Southee,PA Patel': 688, 'CR Brathwaite': 689, 
                                   'Z Khan,KK Nair,Q de Kock': 690, 'Shakib Al Hasan,RV Uthappa': 691, 'KD Karthik,DJ Bravo': 692, 
                                   'MC Henriques,NV Ojha': 693, 'AD Russell,RV Uthappa': 694, 'Harbhajan Singh,PA Patel': 695, 
                                   'KH Pandya': 696, 'P Negi,CH Morris,Q de Kock': 697, 'SS Iyer,Q de Kock': 698, 
                                   'Mustafizur Rahman': 699, 'NS Naik': 700, 'DA Miller,NS Naik': 701, 'SS Tiwary,MS Dhoni': 702, 
                                   'JC Buttler': 703, 'PV Tambe': 704, 'BB Sran': 705, 'KW Richardson,KL Rahul': 706, 
                                   'GJ Maxwell,MP Stoinis': 707, 'JP Faulkner,KD Karthik': 708, 'Ishan Kishan': 709, 
                                   'KC Cariappa': 710, 'RR Pant': 711, 'SW Billings,Q de Kock': 712, 
                                   'GJ Maxwell,Sandeep Sharma': 713, 'R Sathish (sub)': 714, 'WP Saha,Sandeep Sharma': 715, 
                                   'R Sathish (sub),RV Uthappa': 716, 'AM Rahane,MS Dhoni': 717, 'SW Billings': 718, 
                                   'V Shankar (sub)': 719, 'SR Watson,STR Binny': 720, 'Z Khan,Q de Kock': 721, 
                                   'DA Miller,WP Saha': 722, 'BB Sran,Yuvraj Singh': 723, 'DJ Hooda,NV Ojha': 724, 
                                   'AT Rayudu,JC Buttler': 725, 'Harbhajan Singh,KH Pandya': 726, 'JJ Bumrah': 727, 
                                   'SR Watson,Iqbal Abdulla': 728, 'UT Khawaja': 729, 'CJ Jordan': 730, 
                                   'SR Watson,Sachin Baby': 731, 'ER Dwivedi': 732, 'AS Rajpoot,RV Uthappa': 733, 
                                   'KK Nair,Q de Kock': 734, 'DL Chahar': 735, 'F Behardien': 736, 
                                   'Shakib Al Hasan,Kuldeep Yadav': 737, 'YS Chahal,KL Rahul': 738, 
                                   'Sachin Baby,KL Rahul': 739, 'Kuldeep Yadav': 740, 'CJ Jordan (sub)': 741, 
                                   'MS Dhoni,AB Dinda': 742, 'N Rana': 743, 'MP Stoinis': 744, 'BA Stokes': 745, 
                                   'B Stanlake': 746, 'Rashid Khan': 747, 'MJ McClenaghan': 748, 'R Dhawan (sub)': 749, 
                                   'CR Woakes': 750, 'RD Chahar': 751, 'C de Grandhomme': 752, 'JJ Roy': 753, 
                                   'V Kohli,KM Jadhav': 754, 'RA Tripathi': 755, 'C de Grandhomme,RV Uthappa': 756, 
                                   'P Negi,KM Jadhav': 757, 'V Shankar (sub),Yuvraj Singh': 758, 'BB McCullum,KD Karthik': 759, 
                                   'K Rabada': 760, 'KK Nair,RR Pant': 761, 'Basil Thampi': 762, 'TS Mills': 763, 'S Badree': 764, 
                                   'Washington Sundar': 765, 'JD Unadkat,BA Stokes': 766, 'SN Thakur': 767, 'SN Thakur,MS Dhoni': 768, 
                                   'SP Jackson,G Gambhir': 769, 'MA Agarwal (sub)': 770, 'LH Ferguson,MS Dhoni': 771, 
                                   'NM Coulter-Nile,RV Uthappa': 772, 'TM Head' 'Ankit Soni': 773, 'Mohammed Siraj': 774, 
                                   'SP Jackson': 775, 'R Powell (sub)': 776, 'SPD Smith,DT Christian': 777, 
                                   'CR Brathwaite (sub)': 778, 'MJ Guptill': 779, 'DA Warner,B Kumar': 778, 'V Shankar': 779, 
                                   'AR Patel,WP Saha': 780, 'A Mishra,Mohammed Shami': 781, 'J Suchith (sub)': 782, 
                                   'SS Iyer,Z Khan': 783, 'Vishnu Vinod': 784, 'AS Rajpoot': 785, 'J Suchith (sub),PA Patel': 786, 
                                   'DT Christian,SN Thakur': 787, 'MA Wood': 788, 'E Lewis': 789, 'AJ Tye': 790, 'RK Singh': 791, 
                                   'B Laughlin': 792, 'Shivam Mavi (sub)': 793, 'K Gowtham': 794, 'Mujeeb Ur Rahman': 795, 
                                   'DJM Short': 796, 'AP Tare (sub)': 797, 'Shivam Mavi': 798, 'MK Tiwary (sub)': 799, 
                                   'TK Curran': 800, 'H Klaasen': 801, 'LE Plunkett': 802, 'Avesh Khan': 803, 'DJ Hooda (sub)': 804, 
                                   'PP Shaw': 805, 'M Markande': 806, 'AD Hales': 807, 'Shubman Gill': 808, 'DJ Willey': 809, 
                                   'RK Singh (sub)': 810, 'AD Nath': 811, 'JPR Scantlebury-Searles': 812, 'M Prasidh Krishna': 813, 
                                   'RD Chahar (sub)': 814, 'Abhishek Sharma': 815, 'MM Ali': 816, 'R Tewatia (sub)': 817, 
                                   'JC Archer': 818, 'DR Shorey (sub)': 819, 'S Dube': 820, 'Rasikh Salam': 821, 
                                   'KK Nair (sub)': 822, 'SO Hetmyer': 823, 'NA Saini': 824, 'P Chopra (sub)': 825, 
                                   'M Ashwin': 826, 'JM Bairstow': 827, 'Mohammad Nabi': 828, 'H Klaasen (sub)': 829, 
                                   'SM Curran': 830, 'S Lamichhane': 831, 'CA Ingram': 832, 'S Midhun': 833, 'S Gopal': 834, 
                                   'KMA Paul': 835, 'Milind Kumar (sub)': 836, 'N Pooran': 837, 'SE Rutherford': 838, 
                                   'R Parag': 839, 'MJ Santner': 840, 'P Simran Singh': 841, 'LS Livingstone': 842, 
                                   'MK Lomror': 843, 'O Thomas': 844, 'S Sandeep Warrier': 845, 'Anmolpreet Singh (sub)': 846, 
                                   'JL Pattinson': 847, 'D Padikkal': 848, 'PK Garg': 849, 'TU Deshpande': 850, 'KL Nagarkoti': 851, 
                                   'KK Ahmed': 852, 'YBK Jaiswal': 853, 'I Udana': 854, 'JDS Neesham': 855, 'CV Varun': 856, 
                                   'RD Gaikwad': 857, 'AT Carey': 858, 'A Nortje': 859, 'Ravi Bishnoi': 860, 'AS Roy (sub)': 861, 
                                   'Abdul Samad': 862, 'Shahbaz Ahmed': 863, 'CJ Green (sub)': 864, 'PK Garg (sub)': 865, 
                                   'SP Goswami (sub)': 866, 'T Natarajan': 867, 'Arshdeep Singh': 868, 'Lalit Yadav (sub)': 869, 
                                   'Kartik Tyagi': 870, 'DR Sams': 871, 'P Negi (sub)': 872, 'A Zampa': 873, 'KMA Paul (sub)': 874, 
                                   'P Dubey': 875})
df.head()

In [ ]:
df['extras_type'] = df['extras_type'].map({'wides': 0, 'legbyes': 1, 'noballs': 2, 'byes': 3, 'penalty': 4})
df.head()

In [ ]:
df['batting_team'] = df['batting_team'].map({'KKR': 0, 'RCB': 1, 'KXIP': 2, 'CSK': 3, 'RR': 4, 'DC': 5, 'MI': 6, 'SRH': 7, 
                                             'KTK': 8, 'PW': 9, 'RPS': 10, 'GL': 11})
df.head()

In [ ]:
df['bowling_team'] = df['bowling_team'].map({'RCB': 0, 'KKR': 1, 'CSK': 2, 'KXIP': 3, 'DC': 4, 'RR': 5, 'MI': 6, 'SRH': 7, 
                                             'KTK': 8, 'PW': 9, 'RPS': 10, 'GL': 11})
df.head()

# Dist Plot

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["city"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["player_of_match"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["venue"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["team1"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["team2"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["toss_winner"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["winner"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["umpire1"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["umpire2"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["batsman"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["non_striker"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["bowler"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["batsman_runs"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["extra_runs"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["total_runs"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["dismissal_kind"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["player_dismissed"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["fielder"])
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
sns.distplot(df["extras_type"])
plt.show()

# Bar Plot

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'venue', data = df).set_title('City with Venue', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'player_of_match', data = df).set_title('City with Player Of Match', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'team1', data = df).set_title('City with Team1', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'team2', data = df).set_title('City with Team2', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'umpire1', data = df).set_title('City With Umpire1', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'umpire2', data = df).set_title('City With Umpire2', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'toss_winner', data = df).set_title('City with Toss Winner', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'city', y = 'toss_decision', data = df).set_title('City with Toss Decision', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team1', y = 'venue', data = df).set_title('Team1 with Venue', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team2', y = 'venue', data = df).set_title('Team2 with Venue', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team1', y = 'umpire1', data = df).set_title('Team1 With Umpire1', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team2', y = 'umpire2', data = df).set_title('Team2 With Umpire2', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team1', y = 'umpire2', data = df).set_title('Team1 With Umpire2', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team2', y = 'umpire1', data = df).set_title('Team2 With Umpire1', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'venue', y = 'umpire1', data = df).set_title('Venue With Umpire1', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'venue', y = 'umpire2', data = df).set_title('Venue With Umpire2', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team1', y = 'toss_winner', data = df).set_title('Team1 With Toss Winner', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team1', y = 'toss_decision', data = df).set_title('Team1 With Toss Decision', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team2', y = 'toss_winner', data = df).set_title('Team2 With Toss Winner', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team2', y = 'toss_decision', data = df).set_title('Team2 With Toss Decision', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'venue', y = 'toss_winner', data = df).set_title('Venue With Toss Winner', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'venue', y = 'toss_decision', data = df).set_title('City with Toss Decision', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'team1', y = 'team2', data = df).set_title('Team1 With Team2', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'batsman_runs', y = 'extra_runs', data = df).set_title('Batsman Runs With Extra Runs', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'batsman_runs', y = 'total_runs', data = df).set_title('Batsman Runs With Total Runs', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'total_runs', y = 'extra_runs', data = df).set_title('Total Runs With Extra Runs', fontsize = 15)

In [ ]:
plt.figure(figsize = (10, 10))
sns.barplot(x = 'is_wicket', y = 'bowler', data = df).set_title('Is Wicket With Bowler', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'winner', y = 'result_margin', data = df).set_title('Winner With Result Margin', fontsize = 15)

In [ ]:
plt.figure(figsize = (10, 10))
sns.barplot(x = 'result', y = 'result_margin', data = df).set_title('Result With Result Margin', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'toss_winner', y = 'winner', data = df).set_title('Toss Winner With Winner', fontsize = 15)

In [ ]:
plt.figure(figsize = (10, 10))
sns.barplot(x = 'toss_decision', y = 'winner', data = df).set_title('Toss Decision With Winner', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'batting_team', y = 'bowling_team', data = df).set_title('Batting Team With Bowling Team', fontsize = 15)

In [ ]:
plt.figure(figsize = (15, 10))
sns.barplot(x = 'toss_winner', y = 'toss_decision', data = df).set_title('Toss Winner With Toss Decision', fontsize = 15)

# Count Plot

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(x = df['total_runs'],hue = df['extra_runs'])

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(x = df['batsman_runs'],hue = df['extra_runs'])

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(x = df['batsman_runs'],hue = df['total_runs'])

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(x = df['extra_runs'],hue = df['batsman_runs'])

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(x = df['batting_team'],hue = df['bowling_team'])

# Box Plot

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'winner', y = 'result_margin', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'team1', y = 'team2', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'result', y = 'result_margin', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'toss_winner', y = 'winner', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'toss_decision', y = 'winner', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'batting_team', y = 'bowling_team', data = df)

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = 'is_wicket', y = 'bowler', data = df)

In [ ]:
plt.figure(figsize = (15,15))
sns.boxplot(x = 'umpire1', y = 'umpire2', data = df)

# Violin Plot

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['toss_winner'], y = df['toss_decision'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['winner'], y = df['toss_decision'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['winner'], y = df['toss_winner'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['is_wicket'], y = df['bowler'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['is_wicket'], y = df['batsman'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['batsman_runs'], y = df['batsman'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['batsman_runs'], y = df['bowler'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['team1'], y = df['winner'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['team2'], y = df['winner'])
plt.show()

In [ ]:
plt.figure(figsize = (15,10))
sns.violinplot(x = df['team1'], y = df['team2'])
plt.show()

# Group By Plot

In [ ]:
df.groupby('is_wicket')['bowler'].value_counts()[0].head(10)

In [ ]:
df.groupby('is_wicket')['batsman'].value_counts()[0].head(10)

In [ ]:
df.groupby('batsman')['batsman_runs'].value_counts()[0].head()

In [ ]:
df.groupby('bowler')['is_wicket'].value_counts()[0].head()

In [ ]:
df.groupby('over')['is_wicket'].value_counts()[0].head()

In [ ]:
df.groupby('bowler')['total_runs'].value_counts()[0].head()

In [ ]:
df.groupby('toss_winner')['toss_decision'].value_counts()[0].head()

In [ ]:
df.groupby('winner')['result_margin'].value_counts()[0].head()

In [ ]:
df.groupby('city')['venue'].value_counts()[0].head(10)

In [ ]:
df.groupby('city')['team1'].value_counts()[0].head()

In [ ]:
df.groupby('city')['team2'].value_counts()[0].head()

# Cross Tab Plot

In [ ]:
pd.crosstab(df['bowler'],df['is_wicket']).style.background_gradient(cmap = 'winter')

In [ ]:
pd.crosstab(df['toss_winner'],df['toss_decision']).style.background_gradient(cmap = 'spring')

In [ ]:
pd.crosstab(df['team1'],df['team2']).style.background_gradient(cmap = 'autumn')

In [ ]:
pd.crosstab(df['result_margin'],df['result']).style.background_gradient(cmap = 'cool')

In [ ]:
pd.crosstab(df['toss_winner'],df['winner']).style.background_gradient(cmap = 'Wistia')

In [ ]:
pd.crosstab(df['toss_decision'],df['winner']).style.background_gradient(cmap = 'bwr')

In [ ]:
pd.crosstab(df['batsman'],df['is_wicket']).style.background_gradient(cmap = 'seismic')

In [ ]:
pd.crosstab(df['batsman'],df['batsman_runs']).style.background_gradient(cmap = 'PRGn')

In [ ]:
pd.crosstab(df['bowler'],df['batsman_runs']).style.background_gradient(cmap = 'PuOr')

In [ ]:
pd.crosstab(df['team2'],df['winner']).style.background_gradient(cmap = 'YlGnBu')

In [ ]:
pd.crosstab(df['team1'],df['team2']).style.background_gradient(cmap = 'winter')

In [ ]:
pd.crosstab(index = [df['toss_decision'], df['team2']], columns = df['toss_winner'])

In [ ]:
pd.crosstab(index = [df['toss_decision'], df['team1']], columns = df['toss_winner'])

# Cat Plot

In [ ]:
sns.catplot(x = "toss_winner", y = "winner", hue = "toss_decision", data = df)

In [ ]:
sns.catplot(x = "result", y = "winner", hue = "toss_decision", data = df)

# Line Plot

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['team1'],df['team2'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['umpire1'],df['umpire2'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['winner'],df['toss_winner'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['winner'],df['toss_decision'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['bowling_team'],df['batting_team'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['batsman'],df['bowler'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['team1'],df['winner'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['team2'],df['winner'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['team1'],df['umpire1'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['team2'],df['umpire1'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['team1'],df['umpire2'])

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(df['team2'],df['umpire2'])

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x = df['team1'], y = df['team2'],marker = '*', linestyle = '--', color = 'red')

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x = df['umpire1'], y = df['umpire2'],marker = '*', linestyle = '--', color = 'red')

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x = df['batsman'], y = df['bowler'],marker = '*', linestyle = '--', color = 'red')

In [ ]:
df = df.dropna()
#print(df)
df.head()

# Training And Testing Data

In [ ]:
X = df.iloc[:, 0].values.reshape(-1, 1)
Y = df.iloc[:, 1].values.reshape(-1, 1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25,  shuffle = True)

# Linear Regression

In [ ]:
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

In [ ]:
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.show()

In [ ]:
from sklearn import linear_model
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

In [ ]:
# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(X, Y)
# Predict
Y_pred = regression_model.predict(X)

# model evaluation
rmse = mean_squared_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

# printing values
print('Slope:' ,regression_model.coef_)
print('Intercept:', regression_model.intercept_)
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)

# Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, Y_test)))

# Confusion Matrix

In [ ]:
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)

In [ ]:
print(classification_report(Y_test, y_pred))

# Random Forest Classifier Model

In [ ]:
#RANDOM FOREST MODEL
random_classifier = RandomForestClassifier(n_estimators=5, random_state=0)
random_classifier.fit(X_train, Y_train)
random_prediction = random_classifier.predict(X_test)

In [ ]:
#Model Accuracy
print(classification_report(Y_test, random_prediction))
print('Model Accuracy: ',accuracy_score(Y_test, random_prediction))

In [ ]:
# create and fit RandomForestClassifier model  
rfc=RandomForestClassifier()
rfc.fit(X_train, Y_train)

In [ ]:
#predict
pred = rfc.predict(X_test)
pred

In [ ]:
rfc_acc= accuracy_score(Y_test, pred)
print('The accuracy score using the RandomForestClassifier (befor resample) is :',rfc_acc)

In [ ]:
print(classification_report(Y_test, pred))

# Decision Tree Classifier

In [ ]:
# create and fit DecisionTreeClassifier model
dtc = DecisionTreeClassifier()
dtc.fit(X_train,Y_train)

In [ ]:
#predict
pred = dtc.predict(X_test)
dtc_acc = accuracy_score(Y_test, pred)
print('The accuracy score with using the decision tree classifier is :',dtc_acc)

In [ ]:
print(classification_report(Y_test, pred))

# K Nearest Neighbor Classifier

In [ ]:
# create and fit KNeighborsClassifier model
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,Y_train)
#predict
pred = knn.predict(X_test)

In [ ]:
#KNN accuracy score

Knn_acc= accuracy_score(Y_test, knn.predict(X_test))
print('The accuracy socre using the KNeighborsClassifier is :',Knn_acc)

In [ ]:
print(classification_report(Y_test, pred))